# A high resolution model of the grapevine leaf morphospace predicts synthetic leaves
## Reanalysis with added data from Algeria germplasm

_______

# IMPORT MODULES

In [ ]:
# import modules and functions

import matplotlib.pyplot as plt
from pylab import *
import math
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.interpolate import interp1d # for interpolating points
from scipy.signal import find_peaks # for peak detection
from sklearn.decomposition import PCA # for principal component analysis
from scipy.optimize import curve_fit # for fitting curves
from scipy.spatial import procrustes # for Procrustes analysis
from os import listdir # for retrieving files from directory
from os.path import isfile, join # for retrieving files from directory
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis # for LDA
from sklearn.metrics import confusion_matrix # for confusion matrix
from scipy.spatial import ConvexHull # for convex hulls
from matplotlib.patches import Rectangle
from matplotlib.path import Path
from numpy.linalg import det # for sampling higher dimensional convex hull
from scipy.stats import dirichlet
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay


# DEFINE FUNCTIONS

In [ ]:
# define functions

def angle_between(p1, p2, p3):
    """
    define a function to find the angle between 3 points anti-clockwise in degrees, p2 being the vertex
    inputs: three angle points, as tuples
    output: angle in degrees
    """
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    deg1 = (360 + math.degrees(math.atan2(x1 - x2, y1 - y2))) % 360
    deg2 = (360 + math.degrees(math.atan2(x3 - x2, y3 - y2))) % 360
    return deg2 - deg1 if deg1 <= deg2 else 360 - (deg1 - deg2)

def rotate_points(xvals, yvals, degrees):
    """"
    define a function to rotate 2D x and y coordinate points around the origin
    inputs: x and y vals (can take pandas dataframe columns) and the degrees (positive, anticlockwise) to rotate
    outputs: rotated and y vals
    """
    angle_to_move = 270 - degrees
    rads = np.deg2rad(angle_to_move)
    
    new_xvals = xvals*np.cos(rads)-yvals*np.sin(rads)
    new_yvals = xvals*np.sin(rads)+yvals*np.cos(rads)
    
    return new_xvals, new_yvals

def interpolation(x, y, number): 
    """
    define a function to return equally spaced, interpolated points for a given polyline
    inputs: arrays of x and y values for a polyline, number of points to interpolate
    ouputs: interpolated points along the polyline, inclusive of start and end points
    """
    distance = np.cumsum(np.sqrt( np.ediff1d(x, to_begin=0)**2 + np.ediff1d(y, to_begin=0)**2 ))
    distance = distance/distance[-1]

    fx, fy = interp1d( distance, x ), interp1d( distance, y )

    alpha = np.linspace(0, 1, number)
    x_regular, y_regular = fx(alpha), fy(alpha)
    
    return x_regular, y_regular

def euclid_dist(x1, y1, x2, y2):
    """
    define a function to return the euclidean distance between two points
    inputs: x and y values of the two points
    output: the eulidean distance
    """
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def detect_landmark(vein,tip_indices,start_ind,end_ind,ref_ind,use_forward=True,use_max=True):
    """
    define a function to find index of farthest/closest point from ref index point for a set of vein coordinates
    inputs: 2D array of vein points, rows are points columns x and y coordinate values,
    indices of tips, start index, end index, reference idex
    if use_forward=True, then should be used on the forward side of the leaf, otherwise the reverse
    if use_max=True, then finds furthest point from ref, otherwise closest
    ouputs: index of the farthest/closest point from reference
    dependencies: uses the funciton euclid_dist
    """
    ref_dist = [] # store distances to end index 
    dist_ind = [] # store indices for each distance
    if use_forward==True:
        for i in range(start_ind+1, end_ind):
            ref_dist.append(euclid_dist(vein[ref_ind,0],vein[ref_ind,1],vein[i,0],vein[i,1]))
            dist_ind.append(i)
    else:
        for i in range(end_ind+1, start_ind):
            ref_dist.append(euclid_dist(vein[ref_ind,0],vein[ref_ind,1],vein[i,0],vein[i,1]))
            dist_ind.append(i)
    if use_max==True:  
        max_dist_ind = ref_dist.index(max(ref_dist))
        pt_ind = dist_ind[max_dist_ind]
    else:
        min_dist_ind = ref_dist.index(min(ref_dist))
        pt_ind = dist_ind[min_dist_ind]
    return pt_ind
    
def internal_landmarks(vein,tip_indices):
    """
    define a function to return indices of internal vein landmarks given vein tip indices
    inputs: 2D array of vein points, rows are points columns x and y coordinate values,
    indices of tips
    ouputs: a list of indices of internal vein landmarks
    dependencies: detect_landmark
    """
    ### FORWARD SIDE OF LEAF
    
    ### PROXIMAL LOBE
    ### POINT B, 1
    ptB_ind = detect_landmark(vein,tip_indices,tip_indices[1],tip_indices[2],tip_indices[2],use_forward=True,use_max=True)
    ### POINT A, 2
    ptA_ind = detect_landmark(vein,tip_indices,tip_indices[0],tip_indices[1],ptB_ind,use_forward=True,use_max=False)
    ### POINT D, 3
    ptD_ind = detect_landmark(vein,tip_indices,tip_indices[2],tip_indices[3],tip_indices[3],use_forward=True,use_max=True)
    ### POINT C, 4
    ptC_ind = detect_landmark(vein,tip_indices,tip_indices[1],tip_indices[2],ptD_ind,use_forward=True,use_max=False)
    ### POINT F, 5
    ptF_ind = detect_landmark(vein,tip_indices,tip_indices[3],tip_indices[4],tip_indices[4],use_forward=True,use_max=True)
    ### POINT E, 6
    ptE_ind = detect_landmark(vein,tip_indices,tip_indices[2],tip_indices[3],ptF_ind,use_forward=True,use_max=False)
    ### POINT G, 7
    ptG_ind = detect_landmark(vein,tip_indices,tip_indices[4],tip_indices[5],tip_indices[5],use_forward=True,use_max=True)

    ### DISTAL LOBE
    ### POINT I, 8
    ptI_ind = detect_landmark(vein,tip_indices,tip_indices[5],tip_indices[6],tip_indices[6],use_forward=True,use_max=True)
    ### POINT H, 9
    ptH_ind = detect_landmark(vein,tip_indices,tip_indices[4],tip_indices[5],ptI_ind,use_forward=True,use_max=False)
    ### POINT K, 10
    ptK_ind = detect_landmark(vein,tip_indices,tip_indices[6],tip_indices[7],tip_indices[7],use_forward=True,use_max=True)
    ### POINT J, 11
    ptJ_ind = detect_landmark(vein,tip_indices,tip_indices[5],tip_indices[6],ptK_ind,use_forward=True,use_max=False)
    ### POINT M, 12
    ptM_ind = detect_landmark(vein,tip_indices,tip_indices[7],tip_indices[8],tip_indices[8],use_forward=True,use_max=True)
    ### POINT L, 13
    ptL_ind = detect_landmark(vein,tip_indices,tip_indices[6],tip_indices[7],ptM_ind,use_forward=True,use_max=False)
    ### POINT N, 14
    ptN_ind = detect_landmark(vein,tip_indices,tip_indices[8],tip_indices[9],tip_indices[9],use_forward=True,use_max=True)
    
    ### MIDVEIN
    ### POINT P, 15
    ptP_ind = detect_landmark(vein,tip_indices,tip_indices[9],tip_indices[10],tip_indices[10],use_forward=True,use_max=True)
    ### POINT O, 16
    ptO_ind = detect_landmark(vein,tip_indices,tip_indices[8],tip_indices[9],ptP_ind,use_forward=True,use_max=False)
    ### POINT R, 17
    ptR_ind = detect_landmark(vein,tip_indices,tip_indices[10],tip_indices[11],tip_indices[11],use_forward=True,use_max=True)
    ### POINT Q, 18
    ptQ_ind = detect_landmark(vein,tip_indices,tip_indices[9],tip_indices[10],ptR_ind,use_forward=True,use_max=False)
    ### POINT T, 19
    ptT_ind = detect_landmark(vein,tip_indices,tip_indices[11],tip_indices[12],tip_indices[12],use_forward=True,use_max=True)
    ### POINT S, 20
    ptS_ind = detect_landmark(vein,tip_indices,tip_indices[10],tip_indices[11],ptT_ind,use_forward=True,use_max=False)

    ### REVERSE SIDE OF LEAF
    
    ### PROXIMAL LOBE
    ### POINT zB
    ptzB_ind = detect_landmark(vein,tip_indices,tip_indices[-2],tip_indices[-3],tip_indices[-3],use_forward=False,use_max=True)
    ### POINT zA
    ptzA_ind = detect_landmark(vein,tip_indices,tip_indices[-1],tip_indices[-2],ptzB_ind,use_forward=False,use_max=False)
    ### POINT zD
    ptzD_ind = detect_landmark(vein,tip_indices,tip_indices[-3],tip_indices[-4],tip_indices[-4],use_forward=False,use_max=True)
    ### POINT zC
    ptzC_ind = detect_landmark(vein,tip_indices,tip_indices[-2],tip_indices[-3],ptzD_ind,use_forward=False,use_max=False)
    ### POINT zF
    ptzF_ind = detect_landmark(vein,tip_indices,tip_indices[-4],tip_indices[-5],tip_indices[-5],use_forward=False,use_max=True)
    ### POINT zE
    ptzE_ind = detect_landmark(vein,tip_indices,tip_indices[-3],tip_indices[-4],ptzF_ind,use_forward=False,use_max=False)
    ### POINT zG
    ptzG_ind = detect_landmark(vein,tip_indices,tip_indices[-5],tip_indices[-6],tip_indices[-6],use_forward=False,use_max=True)

    ### DISTAL LOBE
    ### POINT zI
    ptzI_ind = detect_landmark(vein,tip_indices,tip_indices[-6],tip_indices[-7],tip_indices[-7],use_forward=False,use_max=True)
    ### POINT zH
    ptzH_ind = detect_landmark(vein,tip_indices,tip_indices[-5],tip_indices[-6],ptzI_ind,use_forward=False,use_max=False)
    ### POINT zK
    ptzK_ind = detect_landmark(vein,tip_indices,tip_indices[-7],tip_indices[-8],tip_indices[-8],use_forward=False,use_max=True)
    ### POINT zJ
    ptzJ_ind = detect_landmark(vein,tip_indices,tip_indices[-6],tip_indices[-7],ptzK_ind,use_forward=False,use_max=False)
    ### POINT zM
    ptzM_ind = detect_landmark(vein,tip_indices,tip_indices[-8],tip_indices[-9],tip_indices[-9],use_forward=False,use_max=True)
    ### POINT zL
    ptzL_ind = detect_landmark(vein,tip_indices,tip_indices[-7],tip_indices[-8],ptzM_ind,use_forward=False,use_max=False)
    ### POINT zN
    ptzN_ind = detect_landmark(vein,tip_indices,tip_indices[-9],tip_indices[-10],tip_indices[-10],use_forward=False,use_max=True)

    ### MIDVEIN
    ### POINT zP
    ptzP_ind = detect_landmark(vein,tip_indices,tip_indices[-10],tip_indices[-11],tip_indices[-11],use_forward=False,use_max=True)
    ### POINT zO
    ptzO_ind = detect_landmark(vein,tip_indices,tip_indices[-9],tip_indices[-10],ptzP_ind,use_forward=False,use_max=False)
    ### POINT zR
    ptzR_ind = detect_landmark(vein,tip_indices,tip_indices[-11],tip_indices[-12],tip_indices[-12],use_forward=False,use_max=True)
    ### POINT zQ
    ptzQ_ind = detect_landmark(vein,tip_indices,tip_indices[-10],tip_indices[-11],ptzR_ind,use_forward=False,use_max=False)
    ### POINT zT
    ptzT_ind = detect_landmark(vein,tip_indices,tip_indices[-12],tip_indices[-13],tip_indices[-13],use_forward=False,use_max=True)
    ### POINT zS
    ptzS_ind = detect_landmark(vein,tip_indices,tip_indices[-11],tip_indices[-12],ptzT_ind,use_forward=False,use_max=False)
    
    landmark_indices = [ptA_ind,ptB_ind,ptC_ind,ptD_ind,ptE_ind,ptF_ind,ptG_ind,ptH_ind,ptI_ind,ptJ_ind,
             ptK_ind,ptL_ind,ptM_ind,ptN_ind,ptO_ind,ptP_ind,ptQ_ind,ptR_ind,ptS_ind,ptT_ind,
             ptzT_ind,ptzS_ind,ptzR_ind,ptzQ_ind,ptzP_ind,ptzO_ind,ptzN_ind,ptzM_ind,ptzL_ind,ptzK_ind,
             ptzJ_ind,ptzI_ind,ptzH_ind,ptzG_ind,ptzF_ind,ptzE_ind,ptzD_ind,ptzC_ind,ptzB_ind,ptzA_ind
            ]
    
    return landmark_indices

def interpolated_intervals(land_indices, new_xvals, new_yvals, num_land):
    
    """
    define a function to return interpolated points for each interval between landmarks of a trace file
    inputs: landmark indices, xvals and yvals of polyline at desired resolution,
    number of interpolated landmarks per interval
    outputs: lists of x and y values with specified number of interpolated points per interval
    dependencies: interpolation
    """
    
    inter_points_x = [] # list to store interpolated x vals
    inter_points_y = [] # list to store interpolated y vals

    for i in range(len(land_indices)-1): # for each index, minus 1, because we are analyzing intervals

        beg_ind = land_indices[i] # specify the beginning point, based on index
        end_ind = land_indices[i+1] # specify the end point, based on index

        interval_xvals = new_xvals[beg_ind:end_ind] # using indices above, find the interval of x vals
        interval_yvals = new_yvals[beg_ind:end_ind] # using indices above, find the interval of y vals

        curr_inter_xvals, curr_inter_yvals = interpolation(interval_xvals, interval_yvals, num_land) # interpolate the interval

        curr_inter_xvals = list(curr_inter_xvals) # convert interval x vals into a list
        curr_inter_yvals = list(curr_inter_yvals) # convert interval y vals into a list

        # to prevent duplicated pseudo-landmark points, because we are working with intervals:

        if i==0: # if the first interval, delete the end point, because it will be covered in the next interval

            del curr_inter_xvals[-1]
            del curr_inter_yvals[-1]

        if i!=0: # if not the first interval, delete the start point, because it was covered in the previous interval

            del curr_inter_xvals[0]
            del curr_inter_yvals[0]

        for j in range(len(curr_inter_xvals)): # for the current interval points

            inter_points_x.append(curr_inter_xvals[j]) # append current interval x vals to list
            inter_points_y.append(curr_inter_yvals[j]) # append current interval y vals to list

    return inter_points_x, inter_points_y

def rotate_and_scale(vein_xvals, vein_yvals, blade_xvals, blade_yvals, base_ind, tip_ind, end_ind, px2_cm2):
    """
    define a function to rotate tip downwards, scale to centimeters, and translate petiolar junction to the origin
    inputs: interpolated x and y vein and blade values, base(first), tip, and end indices, and px2 to cm2 scale
    outputs: rotated, scaled, and translated landmark, vein, and blade coordinates
    dependencies: PCA from sklearn, rotate_points
    """
    vein_arr = np.column_stack((vein_xvals, vein_yvals)) # create vein coordinates array
    blade_arr = np.column_stack((blade_xvals, blade_yvals)) # create blade coordinates array

    vein_len = np.shape(vein_arr)[0] # get lengths of vein and blade arrays to retrieve coords later
    blade_len = np.shape(blade_arr)[0]
    overall_len = vein_len + blade_len

    overall_arr = np.row_stack((vein_arr, blade_arr)) # stack vein and blade arrays into single array

    px_cm = np.sqrt(px2_cm2) # take square root of scaling factor to scale pixels to cm
    scaled_arr = overall_arr/px_cm # convert pixels into cm
    tip_to_base_cm = np.sqrt((scaled_arr[tip_ind,0]-scaled_arr[base_ind,0])**2  # get distance in pixels from tip to base of leaf
                             + (scaled_arr[tip_ind,1]-scaled_arr[base_ind,1])**2) # use to scale PC values back to cm later
    
    # perform a principal component analysis on data to center
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(overall_arr)
    df = pd.DataFrame(data = principalComponents, columns = ['pc1', 'pc2'])
    
    # find the angle of the leaf tip relative to the origin
    
    p1 = (df["pc1"].loc[tip_ind,], df["pc2"].loc[tip_ind,]) # get leaf tip PC1/PC2 coordinate value
    p2 = (0,0) # find angle relative to vertex at origin
    p3 = (10,0) # an arbitrary positive point along the x axis to find angle in anticlockwise direction

    angle = angle_between(p1, p2, p3) # find the angle in degrees of tip point relative to origin, anticlockwise

    rotated_xvals, rotated_yvals = rotate_points(df["pc1"], df["pc2"], angle)

    rotated_arr = np.column_stack((rotated_xvals, rotated_yvals)) # stack x and y vals back into one array

    tip_to_base_pca = np.sqrt((rotated_arr[tip_ind,0]-rotated_arr[base_ind,0])**2 # find the distance in PC vals between tip and base
                              + (rotated_arr[tip_ind,1]-rotated_arr[base_ind,1])**2)

    scale = tip_to_base_cm/tip_to_base_pca # find the factor to scale back to cm

    scaled_arr = rotated_arr*scale # scale rotated PC vals back to cm
    
    pet_junc = np.mean(scaled_arr[[base_ind,end_ind],:],axis=0)
    
    trans_x = scaled_arr[:,0] - pet_junc[0]
    trans_y = scaled_arr[:,1] - pet_junc[1]
    
    scaled_arr = np.column_stack((trans_x, trans_y))
    
    if scaled_arr[10,0] < 0: # insure left side of the leaf is left so labels are on right side of plot
        scaled_arr[:,0] = -scaled_arr[:,0]

    scaled_vein = scaled_arr[0:vein_len,] # isolate just vein coords
    scaled_blade = scaled_arr[vein_len:(vein_len+blade_len),] # isolate just blade coords

    return scaled_vein, scaled_blade # return scaled and rotated vein and blade

def PolyArea(x,y):
    """
    define a function to calculate the area of a polygon using the shoelace algorithm
    inputs: separate numpy arrays of x and y coordinate values
    outputs: the area of the polygon
    """
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

def calc_gpa_mean(shape_arr, num_pseuds, num_coords):
    """
    input: a 3D array of shapes, shapes x num_pseuds landmarks x num_coords coordinates
    output: a generalized Procrustes mean
    """
    
    shape_list = shape_arr # using an array instead of list, rename variable input
    
    ref_ind = 0 # select a reference index to calculate procrustes distances to
    ref_shape = shape_list[ref_ind] # select the reference shape

    mean_diff = 10**(-30) # set a distance between means to stop the algorithm

    old_mean = ref_shape # for the first comparison between means, set old_mean to an arbitrary reference shape

    d = 1000000 # set d initially arbitraily high

    while d > mean_diff: # set boolean criterion for Procrustes distance between mean to stop calculations

        arr = np.zeros( ((len(shape_list)),num_pseuds,num_coords) ) # empty 3D array: # samples, 21 landmarks, 2 coord vals

        for i in range(len(shape_list)): # for each leaf shape after removing outliers

            s1, s2, distance = procrustes(old_mean, shape_list[i]) # calculate procrustes adjusted shape to ref for current leaf
            arr[i] = s2 # store procrustes adjusted shape to array

        new_mean = np.mean(arr, axis=(0)) # calculate mean of all shapes adjusted to reference

        s1, s2, d = procrustes(old_mean, new_mean) # calculate procrustes distance of new mean to old mean

        old_mean = new_mean # set the old_mean to the new_mea before beginning another iteration

    gpa_mean = new_mean
    
    return gpa_mean


# CREATE LIST OF LEAF IDENTIFIERS

For each leaf, starting on one side of the petiolar junction, in ImageJ/Fiji (Schindelin et al., 2012; Schneider et al., 2012) a trace of the vein and blade was made using the line segment tool and stored as a .txt file. An information file, storing species/variety name, the dataset the leaf scan originated from, the vine, developmental stage, file name of the associated image, dataset source, and scaling information, was stored as a .csv file. All three files have a three digit identifier indicating if they are from the Wolfskill/Geneva dataset (starting with "0") or the UC Davis vineyard dataset traced by MSU (starting with "1") or UC Davis (starting with "2) students seprated by underscore and followed by the species/variety name in capital letters. Another underscore separates an identifier indicating if the file is a blade or vein trace or information file ("blade", "vein", and "info", respectively). File names from a data folder were read in and used to store raw data and assocaited labels using the file names.

In [ ]:
data_dir = "./data" # set data directory

file_names = [f for f in listdir(data_dir) if isfile(join(data_dir, f))] # create a list of file names

file_names.remove('.DS_Store') # remove .DS_Store file

file_names.sort() # sort the list of file names

duplicate_leaf_list = [] # store leaf identifiers, they will be duplicated
for i in range(len(file_names)): # for each file
    last_us = file_names[i].rindex("_") # get the last underscore position
    leaf_name = file_names[i][:last_us] # get the leaf identifier
    duplicate_leaf_list.append(leaf_name) # store leaf ID in duplicate_leaf_list
    
leaf_list = [] # a non-duplicated list of leaf identifiers
for i in duplicate_leaf_list: # for each leaf ID in the duplicated list
    if i not in leaf_list: # if it is not already in the unique list
        leaf_list.append(i) # append to the unique list
        
label_list = [] # create a label_list of leaf classes

for i in range(len(leaf_list)):

    leaf_name = leaf_list[i] # get the current leaf name

    first_us = file_names[i].index("_") # get the first underscore position
    leaf_ID = leaf_name[:first_us] # get just the 3 numeral ID of the leaf name
    leaf_num = int(leaf_ID) # convert the leaf_ID into an int
    first_num = leaf_ID[0] # get first number (string) of leaf_ID

    # IDENTIFY ORIGINAL VINIFERA AND WILD LEAVES
    if first_num=="0" and leaf_num==0:
        if leaf_name[4]!="V":
            label_list.append("vinifera")
        elif leaf_name[4]=="V":
            label_list.append("wild")
            
    # IDENTIFY ORIGINAL LEAVES HIGH ERROR 
    elif first_num=="0" and (leaf_num>0 and leaf_num<=17):
        label_list.append("error")
        
    # IDENTIFY ORIGINAL LEAVES USED FOR DEVELOPMENTAL SERIES
    elif first_num=="0" and (leaf_num>=18 and leaf_num<=57):
        if "leaf1" in leaf_name:
            label_list.append("leaf1")
        elif "leaf2" in leaf_name:
            label_list.append("leaf2")
        elif "leaf3" in leaf_name:
            label_list.append("leaf3")
        elif "leaf4" in leaf_name:
            label_list.append("leaf4")
    
    # IDENTIFY MICHIGAN STATE UNIVERSITY LEAVES
    elif first_num=="1":
        label_list.append("MSU")
    
    # IDENTIFY UNIVERSITY OF CALIFORNIA DAVIS LEAVES
    elif first_num=="2":
        label_list.append("UC Davis")

    # IDENTIFY ALGERIA LEAVES
    elif first_num=="3":
        label_list.append("Algeria")

            
    else:
        label_list.append("NA")



# print out leaf and label lists to check
for i in range(len(leaf_list)):
    print(label_list[i], leaf_list[i])
    

# FIND LANDMARKS

90 landmarks (25 from the vein tips, 40 internal vein landmarks, and 25 blade landmarks) were automatically detected from the vein and blade trace files. Vein and blade traces were interpolated with 1000 equidistantly spaced points to increase the resolution of the data using the interp1d function SciPy (Virtanen et al., 2020). The petiolar junction was calculated as the mean of the first ("a) and last ("y") coordinate in the vein trace file. The Euclidean distance of each vein trace point to the petiolar junction was calculated as a function of geodesic distance along the trace itself (or the index of each point, as points are equidistantly placed along the trace). The SciPy find_peaks function was used to find the 25 vein tip indices in the trace using the above plot. The 25 blade landmarks correspond closely to the 25 vein tip landmarks, and are identified by index as the closest point in the interpolated blade trace to an identified vein tip landmark using Euclidean distance. The 25 vein tip landmarks are labeled "a" through "y" in Figure 1A.

40 internal landmarks, placed on each side of the base of a vein, were calculated in the order indicated on Figure 1A. The landmark on the more distal side (farthest away from the petiolar junction) of the base of a vein was calculated before the proximal side (closer to the petiolar junction). The first calculated internal landmark of the base of a vein was used in reference to calculate the other. Internal landmarks were calculated by specifying a start and end index along a segment of the vein trace and a reference index. Calculating the Euclidean distance for each point along the segment from start to end to the reference index, the index with either a maximum or minimum distance value was assigned as the landmark. For example, internal landmark "1" was calculated using landmark "b" as the start index, landmark "c" as the end index, and landmark "c" as the reference index. Internal landmark "1 was calculated as the index with the maximum Euclidean distance along this segment from landmark "c". Landmark 2 was calculated using landmark "a" as the start index, landmark "b" as the end index, and internal landmark "1" as the reference index. Internal landmark "2" was calculated as the index with the minimum Euclideaa distance along this segment from landmark "1". The rest of the internal landmarks were calculated in a similar way, specifying the segment of the trace using previously calculated landmarks on which they fall, and calculating the minimum or maximum Euclidean distance from a reference landmark.

In [ ]:
res = 1000 # set interpolated resolution
dist = 5 # minimum distance between peaks, MODIFY THIS IF YOU ARE HAVING TROUBLE IDENTIFYING VEIN TIPS

tip_indices = [] # tip indices, 25 pts

# NOTE: internal landmark points rely on accurately detecting vein tip indices
land_indices = [] # internal landmarks, 40 pts

# NOTE: blade landmark points rely on accurately detecting vein tip indices
blade_indices = [] # blade indices, 25 pts

for i in range(len(leaf_list)): # for the number of leaves
    
    ### get vein data
    curr_leaf = leaf_list[i] # select current leaf
    print(i,curr_leaf) # PRINT IN CASE OF ERROR TO GO BACK TO PROBLEM LEAVES
    vein = np.loadtxt("./data/" + curr_leaf + "_veins.txt") # load vein trace outline
    inter_vein_x, inter_vein_y = interpolation(vein[:,0],vein[:,1],res) # create interpolated vein outline
    origin = np.mean((vein[0],vein[-1]), axis=0) # find petiolar junction/origin
    
    ### find tip landmarks
    dist_ori = [] # store distance of each interpolated point to the petiolar junction/origin
    for j in range(res): # for the number of interpolated points
        dist_ori.append( euclid_dist(origin[0],origin[1],inter_vein_x[j],inter_vein_y[j]) )
    peaks, _ = find_peaks(dist_ori , height=0, distance=dist) # find peaks/vein tips
    peaks = np.insert(peaks,0,0)
    peaks = np.append(peaks, res-1)
    tip_indices.append(peaks) # save peak indices
    
    ### find internal landmarks
    inter_vein = np.column_stack((inter_vein_x, inter_vein_y))
    landmark_indices = internal_landmarks(inter_vein,peaks)
    land_indices.append(landmark_indices)
    
    ### get blade data
    blade = np.loadtxt("./data/" + curr_leaf + "_blade.txt") # load blade trace outline
    inter_blade_x, inter_blade_y = interpolation(blade[:,0],blade[:,1],res) # create interpolated blade outline
    
    ### find blade landmarks
    blade_pts = [] # to store blade index points
    for k in range(len(peaks)): # for each tip landmark, find closest blade index
        blade_dists = [] # empty list to store distances
        for l in range(res): # for each blade point
            blade_dists.append(euclid_dist(inter_vein_x[peaks[k]],inter_vein_y[peaks[k]],inter_blade_x[l],inter_blade_y[l]))
        blade_pts.append(blade_dists.index(min(blade_dists)))
    blade_indices.append(blade_pts)


# CALCULATE PSEUDOLANDMARKS

Along each line segment of the vein trace defined by internal and tip landmarks, and each line segment of the blade trace defined by blade landmarks, 20 equidistant pseudo-landmarks were calculated. Redundant landmarks at the beginning and end of each segment were eliminated. There were 1672 pseudolandmarks total: 1216 vein and 456 blade.

In [ ]:
# make a list of indices of leaves causing errors
error_indices = [ 35,  # 35 012_leafd_K1163A
                  94,  # 94 112_VITISACERIFOLIA
                  99,  # 99 117_GRN3
                  102, # 102 201_MALVASIABIANCA
                  105, # 'TREBBIANO', this leaf has a duplicate
                  120, # 120 219_DOGRIDGE
                  129, # 129 228_BARBERA
                  139, # 139 238_ALICANTEBOUSCHET
                  140, # 140 239_MARSANNE
                  141, # 141 240_RIPARIAGLOIRE
                  142, # 142 241_RIPARIAGLOIRE
                  145, # 145 244_CILIEGIOLO
                  146, # 146 245_CILIEGIOLO
                  152, # 152 300_AHMEUR BOU AHMEUR_ID01
                  154, #302_AHMEUR BOU AHMEUR_ID03
                  158, #306_AHMEUR BOU AHMEUR_ID07
                  160, #308_AHMEUR BOU AHMEUR_ID09
                  166, #314_BABARI_ID15
                  169, #317_BABARI_ID18
                  181, #329_GENOTYPE 1_ID30
                  189, #337_GENOTYPE 2_ID38
                  190, #338_GENOTYPE 2_ID39
                  196, #344_GENOTYPE 3_ID45 
                  198, #346_GENOTYPE 3_ID47
                  202, #350_GENOTYPE 3_ID51
                  212, #360_GENOTYPE 4_ID61
                  217, #365_GENOTYPE 4_ID66
                  235, #383_TIZI OUININE_ID84
                  237, #385_TIZI OUININE_ID86
                  239, #387_TIZI OUININE_ID88
                ]

print("Out of", len(leaf_list), "original leaves", len(leaf_list)-len(error_indices), "were used")

In [ ]:
num_land = 20 # number of pseudolandmarks to interpolate each line segment
# res is defined above

scaled_vein_list = [] # store final vein pseudolandmarks
scaled_blade_list = [] # store final blade pseudolandmarks
index_list = [] # save indices without error leaves
species_list = [] # save species names for plotting
node_list = [] # save nodes for plotting

for i in range(len(leaf_list)): # for each leaf
    
    curr_leaf = leaf_list[i] # get current leaf
    print(i, curr_leaf)
    if i in error_indices: # skip this leaf if causing errors
        continue

    # read in data from ./data folder
    vein_trace = np.loadtxt("./data/" + curr_leaf + "_veins.txt") # vein trace outline
    inter_vein_x, inter_vein_y = interpolation(vein_trace[:,0],vein_trace[:,1],res) # create high res vein outline
    
    blade_trace = np.loadtxt("./data/" + curr_leaf + "_blade.txt") # blade trace outline
    inter_blade_x, inter_blade_y = interpolation(blade_trace[:,0],blade_trace[:,1],res) # create high res vein outline
    
    info_df = pd.read_csv("./data/" + curr_leaf + "_info.csv") # read in metadata

    # unpack metadata

    if label_list[i]=="Algeria": # if current leaf is Algeria
        species = curr_leaf[:-5] # then use the current leaf name for species
        species_list.append(species)
        node_list.append(leaf)

    else:
        species = info_df.iloc[0,1] 
        species_list.append(species)
        dataset = info_df.iloc[1,1]
        vine = info_df.iloc[2,1]
        leaf = info_df.iloc[3,1]
        node_list.append(leaf)
        image = info_df.iloc[4,1]
        source = info_df.iloc[5,1]
        px2_cm2 = float(info_df.iloc[6,1])
    
    # retrieve landmark data
    
    curr_tip_ind = tip_indices[i] # get tip indices of current leaf
    curr_int_ind = land_indices[i] # get internal indices of current leaf
    curr_bla_ind = blade_indices[i] # get blade indices of current leaf
    
    curr_vei_ind = [ # combine internal and tip indices into vein indices
        
                    curr_tip_ind[0], # beginning petiolar base
        
                    # left proximal lobe
                    curr_int_ind[0],
                    curr_tip_ind[1], # 1st proximal tip
                    
                    curr_int_ind[1],
                    curr_int_ind[2],
                    curr_tip_ind[2], # 2nd proximal tip
                    
                    curr_int_ind[3],
                    curr_int_ind[4],
                    curr_tip_ind[3], # 3rd proximal tip
        
                    curr_int_ind[5],
                    curr_tip_ind[4], # PROXIMAL TIP
        
                    # left distal lobe
                    curr_int_ind[6],
                    curr_int_ind[7],
                    curr_tip_ind[5], # 1st distal tip
                    
                    curr_int_ind[8],
                    curr_int_ind[9],
                    curr_tip_ind[6], # 2nd distal tip
                    
                    curr_int_ind[10],
                    curr_int_ind[11],
                    curr_tip_ind[7], # 3rd distal tip
        
                    curr_int_ind[12],
                    curr_tip_ind[8], # DISTAL TIP
        
                    # left midvein
                    curr_int_ind[13],
                    curr_int_ind[14],
                    curr_tip_ind[9], # 1st midvein tip
                    
                    curr_int_ind[15],
                    curr_int_ind[16],
                    curr_tip_ind[10], # 2nd midvein tip
                    
                    curr_int_ind[17],
                    curr_int_ind[18],
                    curr_tip_ind[11], # 3rd midvein tip
        
                    curr_int_ind[19],
                    curr_tip_ind[12], # MIDVEIN TIP
        
                    # right midvein
                    curr_int_ind[20],
                    curr_tip_ind[13], # 3rd midvein tip
        
                    curr_int_ind[21],
                    curr_int_ind[22],
                    curr_tip_ind[14], # 2nd midvein tip
        
                    curr_int_ind[23],
                    curr_int_ind[24],
                    curr_tip_ind[15], # 1st midvein tip
        
                    curr_int_ind[25],
                    curr_int_ind[26],
        
                    # right distal lobe
                    curr_tip_ind[16], # DISTAL TIP
        
                    curr_int_ind[27],
                    curr_tip_ind[17], # 3rd distal tip
        
                    curr_int_ind[28],
                    curr_int_ind[29],
                    curr_tip_ind[18], # 2nd distal tip
        
                    curr_int_ind[30],
                    curr_int_ind[31],
                    curr_tip_ind[19], # 1st distal tip
        
                    curr_int_ind[32],
                    curr_int_ind[33],
        
                    # right proximal lobe
                    curr_tip_ind[20], # PROXIMAL TIP
        
                    curr_int_ind[34],
                    curr_tip_ind[21], # 3rd proximal tip
        
                    curr_int_ind[35],
                    curr_int_ind[36],
                    curr_tip_ind[22], # 2nd proximal tip
        
                    curr_int_ind[37],
                    curr_int_ind[38],
                    curr_tip_ind[23], # 1st proximal tip
        
                    curr_int_ind[39],
        
                    curr_tip_ind[24] # ending petiolar base
                    ]
    
    vein_pseudx, vein_psuedy = interpolated_intervals(curr_vei_ind, inter_vein_x, inter_vein_y, num_land)
    blade_pseudx, blade_psuedy = interpolated_intervals(curr_bla_ind, inter_blade_x, inter_blade_y, num_land)
    
    scaled_vein, scaled_blade = rotate_and_scale(vein_pseudx, vein_psuedy, 
                                                 blade_pseudx, blade_psuedy, 
                                                 base_ind=0, 
                                                 tip_ind=int(1216/2), 
                                                 end_ind=int(len(vein_pseudx)+len(blade_pseudx))-1, 
                                                 px2_cm2=px2_cm2)
    
    scaled_vein_list.append(scaled_vein)
    scaled_blade_list.append(scaled_blade)
    index_list.append(i)


# VEIN-TO-BLADE RATIO

The ratio of the vein area to the blade area (vein-to-blade ratio) was calculated using Gauss’ area formula (or, the shoelace algorithm; Meister, 1769). The vein and blade pseudo-landmarks were treated as vertices of a polygon from which area was calculated, which were used as the area of the veination and leaf, respectively. The blade area was calcualted by subtracting the vein area from the leaf area. The natural log of vein-to-blade ratio was plotted against the natural log of leaf area to which a line was fitted using the SciPy curve_fit function.

In [ ]:
vtb_min = -4 # set value to normalize vein to blade ratio color
#cmap = cm.get_cmap('plasma', 100) # set the color map, deprecated
cmap = matplotlib.colormaps.get_cmap('plasma')

leaf_areas = [] # store leaf areas
vtb_ratios = [] # store vein-to-blade ratios
hex_colors = [] # store hex colors
lab_colors = [] # store label colors

for i in range(len(scaled_vein_list)): # for each leaf
        
    curr_veins = scaled_vein_list[i] # get vein landmarks
    curr_blade = scaled_blade_list[i] # get blade landmarks
    
    leaf_area = PolyArea(curr_blade[:,0], curr_blade[:,1]) # get leaf area
    vein_area = PolyArea(curr_veins[:,0], curr_veins[:,1]) # get vein area
    blade_area = leaf_area - vein_area # get blade area
    
    vtb_ratio = np.log(vein_area/blade_area) # calculate vein-to-blade ratio
    vtb_normal = vtb_ratio/vtb_min # normalize vein-to-blade ratio
    hexcol = matplotlib.colors.rgb2hex(cmap(vtb_normal)) # get hexcolor
    
    label = label_list[index_list[i]] # get leaf label
    if label=="vinifera":
        lab_col = "#7570b3" # purple
    elif label=="wild":
        lab_col = "#fb6a4a" # lightest red
    elif label=="error":
        lab_col = "#e6ab02" # gold
    elif label=="MSU":
        lab_col = "#66c2a5" # dark green
    elif label=="UC Davis":
        lab_col = "#a6d854" # light green
    elif label=="leaf1":
        lab_col = "#67000d" # darkest red
    elif label=="leaf2":
        lab_col = "#a50f15"
    elif label=="leaf3":
        lab_col = "#cb181d"
    elif label=="leaf4":
        lab_col = "#ef3b2c" # light red
    elif label=="Algeria":
        lab_col = "#013220" # dark green
    
    leaf_areas.append(leaf_area) # store data
    vtb_ratios.append(vtb_ratio)
    hex_colors.append(hexcol)
    lab_colors.append(lab_col)

In [ ]:
# calculate a linear model of ln(vtb) vs ln(leaf area)

def line_func(xvals, m, b):
    return m*xvals + b

popt, pcov = curve_fit(line_func, np.log(leaf_areas), vtb_ratios)

# calculate and visualize residuals for error detection
vtb_resids = line_func(np.log(leaf_areas), popt[0], popt[1]) - np.array(vtb_ratios)

plt.hist(vtb_resids, bins=40, stacked=True, density=True)

In [ ]:
plt.figure(figsize=(8,10))

plt.scatter(np.log(leaf_areas), vtb_ratios, c=lab_colors, s=50, zorder=2, alpha=0.7)
plt.plot(np.log(leaf_areas), line_func(np.log(leaf_areas), popt[0], popt[1]), c="k")
plt.xlabel("ln(leaf area)")
plt.ylabel("ln(vein area / blade area)")
plt.grid()

plt.savefig("./images/Fig01B_allometry_vein_to_blade.png", bbox_inches='tight')


# PROCRUSTES ANALYSIS

The mean Procrustes distance of each leaf to every other leaf was calculated using the procrustes function from SciPy and plotted as a histogram by class. The Generalized Procrustes Analysis mean leaf was calculated by selecting an arbitrary reference leaf as the mean, superimposing all leaves to the reference and calculating a new mean, and continuing the cycle until the difference between two means from two successive iterations fell below an arbitrarily small value (Gower, 1975). For superimposed Procrustes coordinates, leaves were aligned with the calculated Generalized Procrustes Analysis mean leaf.

In [ ]:
# create shape array

num_samples = len(scaled_vein_list)
num_vein_coords = np.shape(scaled_vein_list)[1]
num_blade_coords = np.shape(scaled_blade_list)[1]
num_pseuds = num_vein_coords + num_blade_coords
num_coords = 2

shape_arr = np.zeros((num_samples, num_pseuds, num_coords))

for i in range(len(scaled_vein_list)):
    
    shape_arr[i] = np.row_stack((scaled_vein_list[i], scaled_blade_list[i]))

np.shape(shape_arr)

In [ ]:
# look at distribution of Procrustes distances for outliers

avg_proc_dists = [] # store the average proc distance of each shape to all others
leaf_labels = [] # store labels for each leaf

for i in range(np.shape(shape_arr)[0]): # for each shape
    
    curr_shape = shape_arr[i] # get current shape
    
    curr_dists = [] # store distances to current shape
    
    for j in range(np.shape(shape_arr)[0]): # for each shape
        
        comp_shape = shape_arr[j] # get shape to compare to
        s1, s2, distance = procrustes(curr_shape, comp_shape) # calculate procrustes distance
        curr_dists.append(distance)
        
    avg_proc_dists.append(np.mean(curr_dists)) # store mean proc distance value
    
    leaf_labels.append(label_list[index_list[i]]) # store label of the leaf
    
    
avg_proc_df = pd.DataFrame({"avg_proc_dist":avg_proc_dists, # create a df with proc dists and labels
                           "label":leaf_labels}
        )
    
label_pal = {"vinifera":"#7570b3", # create color palette for seaborn plot for labels
             "wild":"#fb6a4a",
             "error":"#e6ab02",
             "MSU":"#66c2a5",
             "UC Davis":"#a6d854",
             "leaf1":"#67000d",
             "leaf2":"#a50f15",
             "leaf3":"#cb181d",
             "leaf4":"#ef3b2c",
             "Algeria":"#013220"
            }

plt.figure(figsize=(8,10))
ax=sns.histplot(data=avg_proc_df, x="avg_proc_dist", hue="label", multiple="stack", palette=label_pal)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

plt.savefig("./images/Fig01C_proc_distance.png", bbox_inches='tight')


In [ ]:
# calculate GPA mean

gpa_mean = calc_gpa_mean(shape_arr, num_pseuds, num_coords)


In [ ]:
# calculate indices of landmark points
# provide colors for fill

######################
# Individual indices #
######################

peta = 0

lp1a = 18
lp1t = lp1a+19
lp1b = lp1t+20

lp2a = lp1b+18
lp2t = lp2a+19
lp2b = lp2t+20

lp3a = lp2b+18
lp3t = lp3a+19
lp3b = lp3t+20

lptip = lp3b+18

###############

ldst = lptip+20

ld1a = ldst+18
ld1t = ld1a+19
ld1b = ld1t+20

ld2a = ld1b+18
ld2t = ld2a+19
ld2b = ld2t+20

ld3a = ld2b+18
ld3t = ld3a+19
ld3b = ld3t+20

ldtip = ld3b+18

###############

lmst = ldtip+20

lm1a = lmst+18
lm1t = lm1a+19
lm1b = lm1t+20

lm2a = lm1b+18
lm2t = lm2a+19
lm2b = lm2t+20

lm3a = lm2b+18
lm3t = lm3a+19
lm3b = lm3t+20

###############
mtip = lm3b+18
###############

rm3b = mtip+19
rm3t = rm3b+20
rm3a = rm3t+19

rm2b = rm3a+18
rm2t = rm2b+20
rm2a = rm2t+19

rm1b = rm2a+18
rm1t = rm1b+20
rm1a = rm1t+19

rmst = rm1a+18

###############

rdtip = rmst+20

rd3b = rdtip+18
rd3t = rd3b+20
rd3a = rd3t+19

rd2b = rd3a+18
rd2t = rd2b+20
rd2a = rd2t+19

rd1b = rd2a+18
rd1t = rd1b+20
rd1a = rd1t+19

rdst = rd1a+18

###############

rptip = rdst+20

rp3b = rptip+18
rp3t = rp3b+20
rp3a = rp3t+19

rp2b = rp3a+18
rp2t = rp2b+20
rp2a = rp2t+19

rp1b = rp2a+18
rp1t = rp1b+20
rp1a = rp1t+19

rpst = rp1a+18

####################
# Indices for fill #
####################

vein_indices = [
    peta, lp1a, lp1t, lp1b, lp2a, lp2t, lp2b, lp3a, lp3t, lp3b, lptip,
    ldst, ld1a, ld1t, ld1b, ld2a, ld2t, ld2b, ld3a, ld3t, ld3b, ldtip,
    lmst, lm1a, lm1t, lm1b, lm2a, lm2t, lm2b, lm3a, lm3t, lm3b, 
    mtip,
    rm3b, rm3t, rm3a, rm2b, rm2t, rm2a, rm1b, rm1t, rm1a, rmst,
    rdtip, rd3b, rd3t, rd3a, rd2b, rd2t, rd2a, rd1b, rd1t, rd1a, rdst,
    rptip, rp3b, rp3t, rp3a, rp2b, rp2t, rp2a, rp1b, rp1t, rp1a, rpst,
    ]

lp1_indices = list(range(lp1a,lp1b+1))
lp2_indices = list(range(lp2a,lp2b+1))
lp3_indices = list(range(lp3a,lp3b+1))
lprox_indices = list(range(peta,lp1a+1))+list(range(lp1b,lp2a+1))+list(range(lp2b,lp3a+1))+list(range(lp3b,ldst+1))

ld1_indices = list(range(ld1a,ld1b+1))
ld2_indices = list(range(ld2a,ld2b+1))
ld3_indices = list(range(ld3a,ld3b+1))
ldist_indices = list(range(ldst,ld1a+1))+list(range(ld1b,ld2a+1))+list(range(ld2b,ld3a+1))+list(range(ld3b,lmst+1))

lm1_indices = list(range(lm1a,lm1b+1))
lm2_indices = list(range(lm2a,lm2b+1))
lm3_indices = list(range(lm3a,lm3b+1))

mid_indices = list(range(lmst,lm1a+1))+list(range(lm1b,lm2a+1))+list(range(lm2b,lm3a+1))+list(range(lm3b,rm3b+1))+list(range(rm3a,rm2b+1))+list(range(rm2a,rm1b+1))+list(range(rm1a,rmst+1))

rm3_indices = list(range(rm3b,rm3a+1))
rm2_indices = list(range(rm2b,rm2a+1))
rm1_indices = list(range(rm1b,rm1a+1))

rdist_indices = list(range(rmst,rd3b+1))+list(range(rd3a,rd2b+1))+list(range(rd2a,rd1b+1))+list(range(rd1a,rdst+1))
rd3_indices = list(range(rd3b,rd3a+1))
rd2_indices = list(range(rd2b,rd2a+1))
rd1_indices = list(range(rd1b,rd1a+1))

rprox_indices = list(range(rdst,rp3b+1))+list(range(rp3a,rp2b+1))+list(range(rp2a,rp1b+1))+list(range(rp1a,rpst+1))
rp3_indices = list(range(rp3b,rp3a+1))
rp2_indices = list(range(rp2b,rp2a+1))
rp1_indices = list(range(rp1b,rp1a+1))

###################
# Colors for fill #
###################

m1_col = "#990000"
m2_col = "#d7301f"
m3_col = "#ef6548"
mid_col = "#fc8d59"

d1_col = "#7a0177"
d2_col = "#ae017e"
d3_col = "#dd3497"
dist_col = "#f768a1"

p1_col = "#0c2c84"
p2_col = "#225ea8"
p3_col = "#1d91c0"
prox_col = "#41b6c4"


In [ ]:
#################################
# plot the mean Procrustes leaf #
#################################

plt.figure(figsize=(20,20))

# blade outline
plt.plot(gpa_mean[num_vein_coords:,0],
         gpa_mean[num_vein_coords:,1], c="k")

# blade fill
plt.fill(gpa_mean[num_vein_coords:,0],
         gpa_mean[num_vein_coords:,1], c="whitesmoke")


plt.scatter(gpa_mean[vein_indices,0],
         gpa_mean[vein_indices,1],
           s=200, edgecolor="k", zorder=5, facecolor="none")

plt.fill(gpa_mean[lp1_indices,0],
         gpa_mean[lp1_indices,1], c=p1_col)

plt.fill(gpa_mean[lp2_indices,0],
         gpa_mean[lp2_indices,1], c=p2_col)

plt.fill(gpa_mean[lp3_indices,0],
         gpa_mean[lp3_indices,1], c=p3_col)

plt.fill(gpa_mean[lprox_indices,0],
         gpa_mean[lprox_indices,1], c=prox_col)

plt.fill(gpa_mean[ld1_indices,0],
         gpa_mean[ld1_indices,1], c=d1_col)

plt.fill(gpa_mean[ld2_indices,0],
         gpa_mean[ld2_indices,1], c=d2_col)

plt.fill(gpa_mean[ld3_indices,0],
         gpa_mean[ld3_indices,1], c=d3_col)

plt.fill(gpa_mean[ldist_indices,0],
         gpa_mean[ldist_indices,1], c=dist_col)

plt.fill(gpa_mean[lm1_indices,0],
         gpa_mean[lm1_indices,1], c=m1_col)

plt.fill(gpa_mean[lm2_indices,0],
         gpa_mean[lm2_indices,1], c=m2_col)

plt.fill(gpa_mean[lm3_indices,0],
         gpa_mean[lm3_indices,1], c=m3_col)

plt.fill(gpa_mean[mid_indices,0],
         gpa_mean[mid_indices,1], c=mid_col)

plt.fill(gpa_mean[rm3_indices,0],
         gpa_mean[rm3_indices,1], c=m3_col)

plt.fill(gpa_mean[rm2_indices,0],
         gpa_mean[rm2_indices,1], c=m2_col)

plt.fill(gpa_mean[rm1_indices,0],
         gpa_mean[rm1_indices,1], c=m1_col)

plt.fill(gpa_mean[rdist_indices,0],
         gpa_mean[rdist_indices,1], c=dist_col)

plt.fill(gpa_mean[rd3_indices,0],
         gpa_mean[rd3_indices,1], c=d3_col)

plt.fill(gpa_mean[rd2_indices,0],
         gpa_mean[rd2_indices,1], c=d2_col)

plt.fill(gpa_mean[rd1_indices,0],
         gpa_mean[rd1_indices,1], c=d1_col)

plt.fill(gpa_mean[rprox_indices,0],
         gpa_mean[rprox_indices,1], c=prox_col)

plt.fill(gpa_mean[rp3_indices,0],
         gpa_mean[rp3_indices,1], c=p3_col)

plt.fill(gpa_mean[rp2_indices,0],
         gpa_mean[rp2_indices,1], c=p2_col)

plt.fill(gpa_mean[rp1_indices,0],
         gpa_mean[rp1_indices,1], c=p1_col)

plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig("./images/Fig01A_GPA_mean_leaf.png", bbox_inches='tight')

# DISPLAY LEAVES

In [ ]:
for i in range(len(species_list)):
    
    name = species_list[i] # get curret name
    
    # replace name if 'AMPELOPSIS GLANDULOSA VAR BREVIPEDUNCULATA'
    # with "AMPELOPSIS GLANDULOSA"
    
    if name == 'AMPELOPSIS GLANDULOSA VAR BREVIPEDUNCULATA':
        name = "AMPELOPSIS GLANDULOSA"
        species_list[i] = name 

In [ ]:
new_node_list = []

for i in range(len(node_list)):

    text = node_list[i] # get node description
    ind = text.index(",") # get position of comma
    node_text = text[:ind] # get node value
    
    new_node_list.append("node: " + node_text)

In [ ]:
################################
# ORIGINAL, ERROR, DEVELOPMENT #
#### color by vein identity ####
################################

cmap = matplotlib.colormaps.get_cmap('plasma')

plt.figure(figsize=(20,20))

for i in range(80): # for each leaf without errors, get the index
    
    if i%10==0:
        print(i)

    curr_leaf = leaf_list[index_list[i]] # get current leaf
    curr_label = leaf_labels[i] # get current label

    # get vein and blade trace data
    vein_trace = scaled_vein_list[i] # vein trace outline
    blade_trace = scaled_blade_list[i] # blade trace outline
    
    plt.subplot(8,10,i+1)
    
   # blade outline
    plt.plot(blade_trace[:,0],
             blade_trace[:,1], c="k", alpha=0.5)

    # blade fill
    plt.fill(blade_trace[:,0],
             blade_trace[:,1], c="whitesmoke")

    plt.fill(vein_trace[lp1_indices,0],
             vein_trace[lp1_indices,1], c=p1_col)

    plt.fill(vein_trace[lp2_indices,0],
             vein_trace[lp2_indices,1], c=p2_col)

    plt.fill(vein_trace[lp3_indices,0],
             vein_trace[lp3_indices,1], c=p3_col)

    plt.fill(vein_trace[lprox_indices,0],
             vein_trace[lprox_indices,1], c=prox_col)

    plt.fill(vein_trace[ld1_indices,0],
             vein_trace[ld1_indices,1], c=d1_col)

    plt.fill(vein_trace[ld2_indices,0],
             vein_trace[ld2_indices,1], c=d2_col)

    plt.fill(vein_trace[ld3_indices,0],
             vein_trace[ld3_indices,1], c=d3_col)

    plt.fill(vein_trace[ldist_indices,0],
             vein_trace[ldist_indices,1], c=dist_col)

    plt.fill(vein_trace[lm1_indices,0],
             vein_trace[lm1_indices,1], c=m1_col)

    plt.fill(vein_trace[lm2_indices,0],
             vein_trace[lm2_indices,1], c=m2_col)

    plt.fill(vein_trace[lm3_indices,0],
             vein_trace[lm3_indices,1], c=m3_col)

    plt.fill(vein_trace[mid_indices,0],
             vein_trace[mid_indices,1], c=mid_col)

    plt.fill(vein_trace[rm3_indices,0],
             vein_trace[rm3_indices,1], c=m3_col)

    plt.fill(vein_trace[rm2_indices,0],
             vein_trace[rm2_indices,1], c=m2_col)

    plt.fill(vein_trace[rm1_indices,0],
             vein_trace[rm1_indices,1], c=m1_col)

    plt.fill(vein_trace[rdist_indices,0],
             vein_trace[rdist_indices,1], c=dist_col)

    plt.fill(vein_trace[rd3_indices,0],
             vein_trace[rd3_indices,1], c=d3_col)

    plt.fill(vein_trace[rd2_indices,0],
             vein_trace[rd2_indices,1], c=d2_col)

    plt.fill(vein_trace[rd1_indices,0],
             vein_trace[rd1_indices,1], c=d1_col)

    plt.fill(vein_trace[rprox_indices,0],
             vein_trace[rprox_indices,1], c=prox_col)

    plt.fill(vein_trace[rp3_indices,0],
             vein_trace[rp3_indices,1], c=p3_col)

    plt.fill(vein_trace[rp2_indices,0],
             vein_trace[rp2_indices,1], c=p2_col)

    plt.fill(vein_trace[rp1_indices,0],
             vein_trace[rp1_indices,1], c=p1_col)
    
    # plot scale 1cm scale bar
    min_x = np.min(blade_trace[:,0])
    min_y = np.min(blade_trace[:,1])
    plt.plot( [min_x,min_x+1], [min_y-0.2,min_y-0.2], c="k", lw=3   )
    
    title = str(species_list[i])
    
    plt.title(title, fontsize=10)
    plt.axis("off")
    plt.gca().set_aspect("equal")
    plt.tight_layout()
    
plt.savefig("./images/Fig02_display_vein_identity.png", bbox_inches='tight')


In [ ]:
####################################
# MICHIGAN AND CALIFORNIA STUDENTS #
###### color by vein identity ######
####################################

cmap = matplotlib.colormaps.get_cmap('plasma')

plt.figure(figsize=(20,20))

for i in range(80,139): # for each leaf without errors, get the index
    
    if i%10==0:
        print(i)

    curr_leaf = leaf_list[index_list[i]] # get current leaf
    curr_label = leaf_labels[i] # get current label

    # get vein and blade trace data
    vein_trace = scaled_vein_list[i] # vein trace outline
    blade_trace = scaled_blade_list[i] # blade trace outline
    
    plt.subplot(8,10,i-79)
    
   # blade outline
    plt.plot(blade_trace[:,0],
             blade_trace[:,1], c="k", alpha=0.5)

    # blade fill
    plt.fill(blade_trace[:,0],
             blade_trace[:,1], c="whitesmoke")

    plt.fill(vein_trace[lp1_indices,0],
             vein_trace[lp1_indices,1], c=p1_col)

    plt.fill(vein_trace[lp2_indices,0],
             vein_trace[lp2_indices,1], c=p2_col)

    plt.fill(vein_trace[lp3_indices,0],
             vein_trace[lp3_indices,1], c=p3_col)

    plt.fill(vein_trace[lprox_indices,0],
             vein_trace[lprox_indices,1], c=prox_col)

    plt.fill(vein_trace[ld1_indices,0],
             vein_trace[ld1_indices,1], c=d1_col)

    plt.fill(vein_trace[ld2_indices,0],
             vein_trace[ld2_indices,1], c=d2_col)

    plt.fill(vein_trace[ld3_indices,0],
             vein_trace[ld3_indices,1], c=d3_col)

    plt.fill(vein_trace[ldist_indices,0],
             vein_trace[ldist_indices,1], c=dist_col)

    plt.fill(vein_trace[lm1_indices,0],
             vein_trace[lm1_indices,1], c=m1_col)

    plt.fill(vein_trace[lm2_indices,0],
             vein_trace[lm2_indices,1], c=m2_col)

    plt.fill(vein_trace[lm3_indices,0],
             vein_trace[lm3_indices,1], c=m3_col)

    plt.fill(vein_trace[mid_indices,0],
             vein_trace[mid_indices,1], c=mid_col)

    plt.fill(vein_trace[rm3_indices,0],
             vein_trace[rm3_indices,1], c=m3_col)

    plt.fill(vein_trace[rm2_indices,0],
             vein_trace[rm2_indices,1], c=m2_col)

    plt.fill(vein_trace[rm1_indices,0],
             vein_trace[rm1_indices,1], c=m1_col)

    plt.fill(vein_trace[rdist_indices,0],
             vein_trace[rdist_indices,1], c=dist_col)

    plt.fill(vein_trace[rd3_indices,0],
             vein_trace[rd3_indices,1], c=d3_col)

    plt.fill(vein_trace[rd2_indices,0],
             vein_trace[rd2_indices,1], c=d2_col)

    plt.fill(vein_trace[rd1_indices,0],
             vein_trace[rd1_indices,1], c=d1_col)

    plt.fill(vein_trace[rprox_indices,0],
             vein_trace[rprox_indices,1], c=prox_col)

    plt.fill(vein_trace[rp3_indices,0],
             vein_trace[rp3_indices,1], c=p3_col)

    plt.fill(vein_trace[rp2_indices,0],
             vein_trace[rp2_indices,1], c=p2_col)

    plt.fill(vein_trace[rp1_indices,0],
             vein_trace[rp1_indices,1], c=p1_col)
    
    # plot scale 1cm scale bar
    min_x = np.min(blade_trace[:,0])
    min_y = np.min(blade_trace[:,1])
    plt.plot( [min_x,min_x+1], [min_y-0.2,min_y-0.2], c="k", lw=3   )
    
    title = str(species_list[i])
    
    plt.title(title, fontsize=10)
    plt.axis("off")
    plt.gca().set_aspect("equal")
    plt.tight_layout()
    
plt.savefig("./images/Fig03_display_vein_identity.png", bbox_inches='tight')


In [ ]:
####################################
#### ALGERIAN GRAPEVINE LEAVES #####
###### color by vein identity ######
####################################

cmap = matplotlib.colormaps.get_cmap('plasma')

plt.figure(figsize=(20,20))

for i in range(139,len(index_list)): # for each leaf without errors, get the index
    
    if i%10==0:
        print(i)

    curr_leaf = leaf_list[index_list[i]] # get current leaf
    curr_label = leaf_labels[i] # get current label

    # get vein and blade trace data
    vein_trace = scaled_vein_list[i] # vein trace outline
    blade_trace = scaled_blade_list[i] # blade trace outline
    
    plt.subplot(8,10,i-138)
    
   # blade outline
    plt.plot(blade_trace[:,0],
             blade_trace[:,1], c="k", alpha=0.5)

    # blade fill
    plt.fill(blade_trace[:,0],
             blade_trace[:,1], c="whitesmoke")

    plt.fill(vein_trace[lp1_indices,0],
             vein_trace[lp1_indices,1], c=p1_col)

    plt.fill(vein_trace[lp2_indices,0],
             vein_trace[lp2_indices,1], c=p2_col)

    plt.fill(vein_trace[lp3_indices,0],
             vein_trace[lp3_indices,1], c=p3_col)

    plt.fill(vein_trace[lprox_indices,0],
             vein_trace[lprox_indices,1], c=prox_col)

    plt.fill(vein_trace[ld1_indices,0],
             vein_trace[ld1_indices,1], c=d1_col)

    plt.fill(vein_trace[ld2_indices,0],
             vein_trace[ld2_indices,1], c=d2_col)

    plt.fill(vein_trace[ld3_indices,0],
             vein_trace[ld3_indices,1], c=d3_col)

    plt.fill(vein_trace[ldist_indices,0],
             vein_trace[ldist_indices,1], c=dist_col)

    plt.fill(vein_trace[lm1_indices,0],
             vein_trace[lm1_indices,1], c=m1_col)

    plt.fill(vein_trace[lm2_indices,0],
             vein_trace[lm2_indices,1], c=m2_col)

    plt.fill(vein_trace[lm3_indices,0],
             vein_trace[lm3_indices,1], c=m3_col)

    plt.fill(vein_trace[mid_indices,0],
             vein_trace[mid_indices,1], c=mid_col)

    plt.fill(vein_trace[rm3_indices,0],
             vein_trace[rm3_indices,1], c=m3_col)

    plt.fill(vein_trace[rm2_indices,0],
             vein_trace[rm2_indices,1], c=m2_col)

    plt.fill(vein_trace[rm1_indices,0],
             vein_trace[rm1_indices,1], c=m1_col)

    plt.fill(vein_trace[rdist_indices,0],
             vein_trace[rdist_indices,1], c=dist_col)

    plt.fill(vein_trace[rd3_indices,0],
             vein_trace[rd3_indices,1], c=d3_col)

    plt.fill(vein_trace[rd2_indices,0],
             vein_trace[rd2_indices,1], c=d2_col)

    plt.fill(vein_trace[rd1_indices,0],
             vein_trace[rd1_indices,1], c=d1_col)

    plt.fill(vein_trace[rprox_indices,0],
             vein_trace[rprox_indices,1], c=prox_col)

    plt.fill(vein_trace[rp3_indices,0],
             vein_trace[rp3_indices,1], c=p3_col)

    plt.fill(vein_trace[rp2_indices,0],
             vein_trace[rp2_indices,1], c=p2_col)

    plt.fill(vein_trace[rp1_indices,0],
             vein_trace[rp1_indices,1], c=p1_col)
    
    # plot scale 1cm scale bar
    min_x = np.min(blade_trace[:,0])
    min_y = np.min(blade_trace[:,1])
    plt.plot( [min_x,min_x+1], [min_y-0.2,min_y-0.2], c="k", lw=3   )
    
    title = str(species_list[i])
    
    plt.title(title, fontsize=10)
    plt.axis("off")
    plt.gca().set_aspect("equal")
    plt.tight_layout()
    
plt.savefig("./images/FigX_Algeria_display_vein_identity.png", bbox_inches='tight')

# BIOLOGICAL REPRODUCIBILITY



To assess the ability of our morphometric method to measure biological reproducibility, we compared the two halves of each leaf to each other as well as a set of 20 pairs of leaves originating from the same variety in the UC Davis collection and calculated Procrustes distances. For each of the two analyses, the average Procrustes distance for all comparisons was first calculated. 1000 bootstrap calculations of the average Procrustes distance were calculated by randomizing one set of the data against the other and comparing the distribution to the actual average Procrustes distance. For each pseudo-landmark from each comparison, a Euclidean distance was calculated and projected back onto Generalized Procrustes Analysis mean leaf to visualize the spatial contribution of each pseudo-landmark to the alignment.

#### Compare procrustes distances between leaf sides

In [ ]:
blade_tip_ind = int(num_vein_coords + num_blade_coords/2) # store the blade tip index
# there are 608 vein coords per a half leaf
# there are 228 blade coords per a half leaf

side_dists = [] # store proc dists between leaf sides
pt_distances = [] # store proc dists for each point

for i in range(np.shape(shape_arr)[0]):

    left_veins = shape_arr[i,0:vein_indices[32]+1,:] # isolate veins left side
    right_veins = shape_arr[i,vein_indices[32]:vein_indices[64],:] # isolate veins right side

    left_blade = shape_arr[i,num_vein_coords:blade_tip_ind,:] # isolate blade left side
    right_blade = shape_arr[i,blade_tip_ind:,:] # isolate blade right side

    left_side = np.row_stack((left_veins, left_blade)) # create left side of leaf
    right_side = np.row_stack((right_veins, right_blade)) # create right side of leaf

    s1, s2, distance = procrustes(left_side, right_side) # calculate procrustes distance between two sides
    side_dists.append(distance)

    pt_dists = [] # store proc distances between each point
    for i in range(len(s1)):
        pt_dists.append(euclid_dist(s1[i,0],s1[i,1],s2[i,0],s2[i,1]))
    pt_distances.append(pt_dists) # append distances for each point
    
pt_dist_arr = np.array(pt_distances) # convert proc dists for each point to array
pt_dist_means = np.mean(pt_dist_arr,0) # get proc dist means for each point
sides_dist_mean = np.mean(side_dists) # get mean of proc dists between sides

print(sides_dist_mean)

#### Visualize procrustes distances between pseudo-landmarks

In [ ]:
plt.figure(figsize=(10,10))

left_veins = gpa_mean[0:vein_indices[32]+1,:] # isolate veins left side mean leaf
left_blade = gpa_mean[num_vein_coords:blade_tip_ind,:] # isolate blade left side mean leaf

left_mean_leaf = np.row_stack((left_veins,left_blade)) # stack vein and blade for mean leaf
left_mean_df = pd.DataFrame({"x_vals":left_mean_leaf[:,0],
                             "y_vals":left_mean_leaf[:,1],
                             "dists":pt_dist_means})
       
ax = sns.scatterplot(data=left_mean_df, x="x_vals", y="y_vals", hue=-np.log(left_mean_df["dists"]), s=50, palette="inferno")
sns.move_legend(ax,"upper left", bbox_to_anchor=(1, 1))
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig("./images/Fig01D_leaf_sides_points.png", bbox_inches='tight')


#### Bootstrap procrustes distances between leaf sides

In [ ]:
blade_tip_ind = int(num_vein_coords + num_blade_coords/2) # store the blade tip index
# there are 608 vein coords per a half leaf
# there are 228 blade coords per a half leaf

rands = 1000 # set number of randomizations
rand_list = [] # store proc distances of randomizations

for r in range(rands):
    
    if r%100==0:
        print(r)

    seq_list = list(range(np.shape(shape_arr)[0])) # create a sequential list to shuffle
    shuffle(seq_list) # shuffle the list

    side_dists = [] # store proc dists between leaf sides

    for i in range(np.shape(shape_arr)[0]):

        left_veins = shape_arr[i,0:vein_indices[32]+1,:] # isolate veins left side
        rand_veins = shape_arr[seq_list[i],vein_indices[32]:vein_indices[64],:] # isolate veins right side

        left_blade = shape_arr[i,num_vein_coords:blade_tip_ind,:] # isolate blade left side
        rand_blade = shape_arr[seq_list[i],blade_tip_ind:,:] # isolate blade right side

        left_side = np.row_stack((left_veins, left_blade)) # create left side of leaf
        rand_side = np.row_stack((rand_veins, rand_blade)) # create right side of leaf

        s1, s2, distance = procrustes(left_side, rand_side) # calculate procrustes distance between two sides
        side_dists.append(distance)

    rand_list.append(np.mean(side_dists)) # get mean of proc dists between sides and store

In [ ]:
# the number of times the mean random procrustes distance was less than the actual
sum(rand_list < sides_dist_mean)

In [ ]:
plt.figure(figsize=(10,10))

sns.histplot(rand_list)
plt.axvline(sides_dist_mean, 0,120, lw=5, c="red")
plt.savefig("./images/Fig01E_leaf_sides_histogram.png", bbox_inches='tight')

#### Compare procrustes distances between leaves from varieties

In [ ]:
# Print out species names and indices to find pairs to compare
for i in range(len(species_list)):
    print(i, species_list[i])

In [ ]:
# create a list of lists with species names and indices to compare
pair_list = [ # a list of lists, each element has SPECIES and two indices of the pair 
    ["VITIS ROMANETII",80,82],
    ["VITIS DOANIANA",81,83],
    ["5C",84,85],
    ["DOG RIDGE",86,87],
    ["CORVINA VERONSE",88,89],
    ["AXR1",90,91],
    ["O39-16",93,94],
    ["CINSAULT",95,96],
    ["BONARDA",103,104],
    ["RS-9",105,106],
    ["VALDEPENAS",108,109],
    ["FOLLE BLANCE",111,112],
    ["PETIT VERDOT",113,114],
    ["RAMSEY",116,117],
    ["PARELLADA",118,119],
    ["44-53M",120,121],
    ["MUSCAT OTTONEL",123,124],
    ["5BB",128,129],
    ["8B",132,133],
    ["VITIS ARIZONICA",135,136]
]

In [ ]:
pair_distances = [] # a list to store the distances between leaf pairs
pt_distances = [] # store proc dists for each point

for i in range(len(pair_list)):
    
    first_leaf = shape_arr[pair_list[i][1],:,:] # isolate first leaf
    second_leaf = shape_arr[pair_list[i][2],:,:] # isolate second leaf

    # calculate procrustes distance between the pair of leaves
    s1, s2, distance = procrustes(first_leaf, second_leaf)
    pair_distances.append(distance) # append distance
    
    pt_dists = [] # store proc distances between each point
    for i in range(len(s1)):
        pt_dists.append(euclid_dist(s1[i,0],s1[i,1],s2[i,0],s2[i,1]))
    pt_distances.append(pt_dists) # append distances for each point
    
pt_dist_arr = np.array(pt_distances) # convert proc dists for each point to array
pt_dist_means = np.mean(pt_dist_arr,0) # get proc dist means for each point

pair_dist_mean = np.mean(pair_distances) # calculate mean distance between pairs
print(pair_dist_mean) # print mean distance between pairs

#### Visualize procrustes distances between pseudo-landmarks for pairs

In [ ]:
plt.figure(figsize=(10,10))

pair_mean_df = pd.DataFrame({"x_vals":gpa_mean[:,0],
                             "y_vals":gpa_mean[:,1],
                             "dists":pt_dist_means})
       
ax = sns.scatterplot(data=pair_mean_df, x="x_vals", y="y_vals", hue=-np.log(pair_mean_df["dists"]), s=50, palette="inferno")
sns.move_legend(ax,"upper left", bbox_to_anchor=(1, 1))
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig("./images/Fig01F_leaf_pairs_points.png", bbox_inches='tight')


#### Bootstrap procrustes distances between pairs of leaves

In [ ]:
rands = 1000 # set number of randomizations
rand_list = [] # store proc distances of randomizations

for r in range(rands):
    
    if r%100==0:
        print(r)

    seq_list = list(np.array(pair_list)[:,2]) # create a sequential list to shuffle
    shuffle(seq_list) # shuffle the list

    pair_distances = [] # store proc dists between leaf pairs
    
    for i in range(len(pair_list)):
    
        first_leaf = shape_arr[pair_list[i][1],:,:] # isolate first leaf
        rand_leaf = shape_arr[int(seq_list[i]),:,:] # isolate random leaf

        # calculate procrustes distance between the pair of leaves
        s1, s2, distance = procrustes(first_leaf, rand_leaf)
        pair_distances.append(distance) # append distance
    
    rand_list.append(np.mean(pair_distances)) # get mean of proc dists between leaf pairs

In [ ]:
# the number of times the mean random procrustes distance was less than the actual
sum(rand_list < pair_dist_mean)

In [ ]:
plt.figure(figsize=(10,10))

sns.histplot(rand_list)
plt.axvline(pair_dist_mean, 0,120, lw=5, c="red")
plt.savefig("./images/Fig01G_leaf_pairs_histogram.png", bbox_inches='tight')

# MODEL LEAF DEVELOPMENT

Developmental modeling was undertaken using the same techniques as Bryson et al. (2020) but with 1672 pseudo-landmarks rather than just 21 landmarks. For 10 wild Vitis spp., using the first four expanded leaves from the growing tip of a single shoot, for each of 3344 x and y coordinate values from 1672 pseudo-landmarks, a second degree polynomial model was fitted as a function of node number counting from the tip (1, 2, 3, 4) using the Numpy polyfit function. 100 modeled leaves were modeled from node 1 to 4 and used in subsequent analyses.

#### Generalized Procrustes Analysis to compare leaves

In [ ]:
# an array to store procrustes aligned shapes
proc_arr = np.zeros((np.shape(shape_arr)[0],
                     np.shape(shape_arr)[1],
                     np.shape(shape_arr)[2]
                    ))
# for each leaf
for i in range(np.shape(shape_arr)[0]):

    # calculate superimposed shape to gpa_mean
    s1, s2, distance = procrustes(gpa_mean, shape_arr[i,:,:]) 
    
    # append procrustes shape to array
    proc_arr[i,:,:] = s2


#### Model leaf development

In [ ]:
for i in range(len(proc_arr)):
    print(i, species_list[i], leaf_labels[i])

In [ ]:
leaf_devo_list = [
["VITIS CINEREA",40,41,42,43],
["VITIS RUPESTRIS",44,45,46,47],
["VITIS RIPARIA",48,49,50,51],
["VITIS LABRUSCA",52,53,54,55],
["VITIS ACERIFOLIA",56,57,58,59],
["VITIS AMURENSIS",60,61,62,63],
["VITIS VULPINA",64,65,66,67],
["VITIS AESTIVALIS",68,69,70,71],
["VITIS COIGNETIAE",72,73,74,75],
["VITIS PALMATA",76,77,78,79]
]

In [ ]:
intervals = 100
poly_order = 2

leaf_model_arr = np.zeros( (len(leaf_devo_list),100,1672,2)    )

for s in range(len(leaf_devo_list)):
    
    print(s, leaf_devo_list[s][0])

    leaf1 = proc_arr[leaf_devo_list[s][1],:,:]
    leaf2 = proc_arr[leaf_devo_list[s][2],:,:]
    leaf3 = proc_arr[leaf_devo_list[s][3],:,:]
    leaf4 = proc_arr[leaf_devo_list[s][4],:,:]

    leaf_arr = np.array((leaf1, leaf2, leaf3, leaf4))

    total_length = len(leaf1) # the number of landmarks to consider

    # an array to hold the  modeled data
    model_arr = np.zeros((intervals,total_length,2))

    for i in range(total_length): # for i landmark 

        for j in range(2): # for j coordinates in x and y, 0 and 1

            # fit function across values 1, 2, 3, and 4 for current i, j landmark coord val
            N = np.polyfit( np.array([1,2,3,4]), leaf_arr[:, i, j], poly_order )
            func = np.poly1d(N)

            # model vals over interval numbers for leaves 1 through 4
            model_vals = func(np.linspace(1,4,intervals))

            # place modeled values into array
            model_arr[0:100, i, j] = model_vals
            
    leaf_model_arr[s,:,:,:] = model_arr


In [ ]:
np.shape(leaf_model_arr)

In [ ]:
# Visualize leaf development

cmap = cm.get_cmap('plasma', 100) # set the color map

overall_counter = 1

plt.figure(figsize=(20,20))

for i in range( np.shape(leaf_model_arr)[0] ): # for species i
    
    print(leaf_devo_list[i][0])
    
    for j in [0,11,22,33,44,55,66,77,88,99]: # for j intervals

        plt.subplot(10,10,overall_counter)

        lf = leaf_model_arr[i,j,:,:]

        lf_veinX = lf[0:num_vein_coords,0]
        lf_veinY = lf[0:num_vein_coords,1]
        lf_bladeX = lf[num_vein_coords:,0]
        lf_bladeY = lf[num_vein_coords:,1]

        vein_area = PolyArea(lf_veinX,lf_veinY)
        leaf_area = PolyArea(lf_bladeX,lf_bladeY)
        blade_area = leaf_area-vein_area
        vtb_ratio = np.log(vein_area/blade_area)
        vtb_min = -4
        norm_vtb = vtb_ratio/vtb_min
        hexcol = matplotlib.colors.rgb2hex(cmap(norm_vtb))

        plt.fill(lf_bladeX, lf_bladeY, c="lightgray")
        plt.plot(lf_bladeX, lf_bladeY, c="k", lw=1)
        plt.fill(lf_veinX, lf_veinY, c=hexcol)
        plt.axis("off")
        plt.gca().set_aspect("equal")

        overall_counter+=1
        
plt.tight_layout

plt.savefig("./images/Fig04A_leaf_development.png", bbox_inches='tight')


# PRINCIPAL COMPONENT ANALYSIS

The PCA function from Scikit-learn (Pedregosa et al., 2011) was used for Principal Component Analysis (PCA). Eigenleaf representations were calculated using the inverse_transform function. 

#### Percent variance for all PCs

In [ ]:
######
PC_NUMBER = 139
#######

# use the reshape function to flatten proc_arr to 2D
reshaped_arr = proc_arr.reshape(np.shape(proc_arr)[0], 
                                      np.shape(proc_arr)[1]*2) 

pca = PCA(n_components=PC_NUMBER) 
PCs = pca.fit_transform(reshaped_arr) # fit a PCA

print(pca.explained_variance_ratio_) # print out explained variance for each PC
print(pca.explained_variance_ratio_.cumsum())


In [ ]:
var_df = pd.DataFrame({"ind":list(range(139)),
                       "var":pca.explained_variance_ratio_,
                       "hue":"var"
                      })

cumul_var_df = pd.DataFrame({"ind":list(range(139)),
                       "var":pca.explained_variance_ratio_.cumsum(),
                       "hue":"cumul_var"
                      })

var_bar_df = pd.concat([var_df,cumul_var_df])

plt.figure(figsize=(20,2))
ax=sns.barplot(var_bar_df, x="ind", y="var", hue="hue")
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.hlines(y=0.57525645, xmin=0, xmax=138)
plt.hlines(y=0.88078701, xmin=0, xmax=138)
plt.xlim(0,138)

In [ ]:
######
PC_NUMBER = 9
#######

# use the reshape function to flatten proc_arr to 2D
reshaped_arr = proc_arr.reshape(np.shape(proc_arr)[0], 
                                      np.shape(proc_arr)[1]*2) 

pca = PCA(n_components=PC_NUMBER) 
PCs = pca.fit_transform(reshaped_arr) # fit a PCA

print(pca.explained_variance_ratio_) # print out explained variance for each PC
print(pca.explained_variance_ratio_.cumsum())

In [ ]:
num_PCs = 9
st_devs = [-2,-1,0,1,2]

PCs_eigenleaves = np.zeros((45,9))

counter=0

for i in range(num_PCs):
    curr_PC = PCs[:,i]
    std_curr_PC = np.std(curr_PC)
    for j in st_devs:
        PC_std_val = std_curr_PC*j
        PCs_eigenleaves[counter,i] = PC_std_val
        counter+=1
        
PCs_eigenleaves

In [ ]:
# Visualize random genotype leaves

plt.figure(figsize=(10,20))

cmap = cm.get_cmap('plasma', 100) # set the color map
counter = 1

for i in range(45):
    
    if i%5==0:
        print(i)

    inv_leaf = pca.inverse_transform(PCs_eigenleaves[i,:])
    inv_leaf = np.reshape(inv_leaf, (1672,2))

    inv_leaf_veinX = inv_leaf[0:num_vein_coords,0]
    inv_leaf_veinY = inv_leaf[0:num_vein_coords,1]
    inv_leaf_bladeX = inv_leaf[num_vein_coords:,0]
    inv_leaf_bladeY = inv_leaf[num_vein_coords:,1]
    
    vein_area = PolyArea(inv_leaf_veinX,inv_leaf_veinY)
    leaf_area = PolyArea(inv_leaf_bladeX,inv_leaf_bladeY)
    blade_area = leaf_area-vein_area
    vtb_ratio = np.log(vein_area/blade_area)
    vtb_min = -4
    norm_vtb = vtb_ratio/vtb_min
    hexcol = matplotlib.colors.rgb2hex(cmap(norm_vtb))

    plt.subplot(9,5,counter)
    plt.plot(inv_leaf_bladeX, inv_leaf_bladeY, c="k", lw=1)
    plt.fill(inv_leaf_bladeX, inv_leaf_bladeY, c="lightgray")
    plt.fill(inv_leaf_veinX, inv_leaf_veinY, c=hexcol, zorder=2)
    plt.gca().set_aspect("equal")
    plt.gca().set_facecolor("white")
    plt.axis("off")

    counter += 1
        
plt.tight_layout()
plt.savefig("./images/Fig04B_eigenleaves.png", bbox_inches='tight')

#### Create a PC model with only 2 PCs for reconstruction of eigenleaves

In [ ]:
######
PC_NUMBER = 2
#######

# use the reshape function to flatten proc_arr to 2D
reshaped_arr = proc_arr.reshape(np.shape(proc_arr)[0], 
                                      np.shape(proc_arr)[1]*2) 

pca = PCA(n_components=PC_NUMBER) 
PCs = pca.fit_transform(reshaped_arr) # fit a PCA

print(pca.explained_variance_ratio_) # print out explained variance for each PC
print(pca.explained_variance_ratio_.cumsum())

In [ ]:
# create pca df to visualize results in seaborn
pca_df = pd.DataFrame({
    "PC1":PCs[:,0],
    "PC2":PCs[:,1],
    "label":leaf_labels})

# create color palette for seaborn plot for labels
label_pal = {"vinifera":"#7570b3", 
             "wild":"#fb6a4a",
             "error":"#e6ab02",
             "MSU":"#66c2a5",
             "UC Davis":"#a6d854",
             "leaf1":"#67000d",
             "leaf2":"#a50f15",
             "leaf3":"#cb181d",
             "leaf4":"#ef3b2c",
             "rootstock":"green",
             "dissected":"gold",
             "Algeria":"#013220"
            }

xlim1 = -0.35
xlim2 = 0.35
ylim1 = -0.15
ylim2 = 0.20

plt.figure(figsize=(10,10))
ax=sns.scatterplot(data=pca_df, x="PC1", y="PC2", hue="label", palette=label_pal, zorder=2, s=70)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.xlim(xlim1,xlim2)
plt.ylim(ylim1,ylim2)
plt.grid()
plt.gca().set_aspect("equal")

In [ ]:
# recode labels as vinifera, wild, rootstock, dissected

geno_labels = ["vinifera",#'ALICANTE BOUSCHET'
 "vinifera",#'CABERNET SAUVIGNON'
 "vinifera",#'CHARDONNAY'
 "dissected",#'CHASSELAS CIOUTAT'
 "vinifera",#'CHASSELAS DORÉ'
 "vinifera",#'FLAME TOKAY'
 "vinifera",#'GEWÜRTZTRAMINER'
 "vinifera",#'PINOT NOIR'
 "vinifera",#'SYLVANER'
 "vinifera",#'SYRAH'
 "wild",#'VITIS ACERIFOLIA'
 "wild",#'VITIS AESTIVALIS'
 "wild",#'VITIS AMURENSIS'
 "wild",#'VITIS CINEREA'
 "wild",#'VITIS COIGNETIAE'
 "wild",#'VITIS LABRUSCA'
 "wild",#'VITIS PALMATA'
 "dissected",#'VITIS PIASEZKII'
 "wild",#'VITIS RIPARIA'
 "wild",#'VITIS RUPESTRIS'
 "wild",#'VITIS THUNBERGII'
 "wild",#'VITIS VULPINA'
 "vinifera",#'WHITE RIESLING'
 "vinifera",#'ZINFANDEL'
 "wild",#'VITIS RUPESTRIS'
 "dissected",#'AMPELOPSIS ACONITIFOLIA'
 "wild",#'VITIS RUPESTRIS'
 "dissected",#'CHASSELAS CIOUTAT'
 "dissected",#'CHASSELAS CIOUTAT'
 "wild",#'VITIS VULPINA'
 "wild",#'VITIS THUNBERGII'
 "dissected",#'AMPELOPSIS GLANDULOSA'
 "dissected",#'AMPELOPSIS GLANDULOSA'
 "wild",#'VITIS VULPINA'
 "dissected",#'VITIS PIASEZKII'
 "dissected",#'AMPELOPSIS ACONITIFOLIA'
 "dissected",#'AMPELOPSIS GLANDULOSA'
 "wild",#'VITIS RIPARIA'
 "wild",#'VITIS RUPESTRIS'
 "dissected",#'AMPELOPSIS ACONITIFOLIA'
 "wild",#'VITIS CINEREA'
 "wild",#'VITIS CINEREA'
 "wild",#'VITIS CINEREA'
 "wild",#'VITIS CINEREA'
 "wild",#'VITIS RUPESTRIS'
 "wild",#'VITIS RUPESTRIS'
 "wild",#'VITIS RUPESTRIS'
 "wild",#'VITIS RUPESTRIS'
 "wild",#'VITIS RIPARIA'
 "wild",#'VITIS RIPARIA'
 "wild",#'VITIS RIPARIA'
 "wild",#'VITIS RIPARIA'
 "wild",#'VITIS LABRUSCA'
 "wild",#'VITIS LABRUSCA'
 "wild",#'VITIS LABRUSCA'
 "wild",#'VITIS LABRUSCA'
 "wild",#'VITIS ACERIFOLIA'
 "wild",#'VITIS ACERIFOLIA'
 "wild",#'VITIS ACERIFOLIA'
 "wild",#'VITIS ACERIFOLIA'
 "wild",#'VITIS AMURENSIS'
 "wild",#'VITIS AMURENSIS'
 "wild",#'VITIS AMURENSIS'
 "wild",#'VITIS AMURENSIS'
 "wild",#'VITIS VULPINA'
 "wild",#'VITIS VULPINA'
 "wild",#'VITIS VULPINA'
 "wild",#'VITIS VULPINA'
 "wild",#'VITIS AESTIVALIS'
 "wild",#'VITIS AESTIVALIS'
 "wild",#'VITIS AESTIVALIS'
 "wild",#'VITIS AESTIVALIS'
 "wild",#'VITIS COIGNETIAE'
 "wild",#'VITIS COIGNETIAE'
 "wild",#'VITIS COIGNETIAE'
 "wild",#'VITIS COIGNETIAE'
 "wild",#'VITIS PALMATA'
 "wild",#'VITIS PALMATA'
 "wild",#'VITIS PALMATA'
 "wild",#'VITIS PALMATA'
 "wild",#'VITIS ROMANETII'
 "wild",#'VITIS DOANIANA'
 "wild",#'VITIS ROMANETII'
 "wild",#'VITIS DOANIANA'
 "rootstock",#'5C'
 "rootstock",#'5C'
 "rootstock",#'DOG RIDGE'
 "rootstock",#'DOG RIDGE'
 "vinifera",#'CORVINA VERONSE'
 "vinifera",#'CORVINA VERONSE'
 "rootstock",#'AXR1'
 "rootstock",#'AXR1'
 "wild",#'VITIS ACERIFOLIA'
 "rootstock",#'O39-16'
 "rootstock",#'O39-16'
 "vinifera",#'CINSAULT'
 "vinifera",#'CINSAULT'
 "rootstock",#'GRN3'
 "vinifera",#'MERLOT'
 "vinifera",#'MALVASIA BIANCA'
 "rootstock",#'1613C'
 "vinifera",#'TREBBIANO'
 "vinifera",#'SYLVANER'
 "vinifera",#'BONARDA'
 "vinifera",#'BONARDA'
 "rootstock",#'RS-9'
 "rootstock",#'RS-9'
 "vinifera",#'CABERNET FRANC'
 "vinifera",#'VALDEPENAS'
 "vinifera",#'VALDEPENAS'
 "vinifera",#'VESPOLINA'
 "vinifera",#'FOLLE BLANCE'
 "vinifera",#'FOLLE BLANCE'
 "vinifera",#'PETIT VERDOT'
 "vinifera",#'PETIT VERDOT'
 "rootstock",#'DOG RIDGE'
 "rootstock",#'RAMSEY'
 "rootstock",#'RAMSEY'
 "vinifera",#'PARELLADA'
 "vinifera",#'PARELLADA'
 "rootstock",#'44-53M'
 "rootstock",#'44-53M'
 "rootstock",#'1103P'
 "vinifera",#'MUSCAT OTTONEL'
 "vinifera",#'MUSCAT OTTONEL'
 "vinifera",#'SIEGERREBE'
 "rootstock",#'5C'
 "rootstock",#'5C'
 "rootstock",#'5BB'
 "rootstock",#'5BB'
 "vinifera",#'TANNAT'
 "vinifera",#'ALICANTE BOUSCHET'
 "rootstock",#'8B'
 "rootstock",#'8B'
 "vinifera",#'EMERALD RIESLING'
 "wild",#'VITIS ARIZONICA'
 "wild",#'VITIS ARIZONICA'
 "rootstock",#'DOG RIDGE'
 "vinifera",#'BURGER'
# the remainder are all algeria
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
"algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
"algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
"algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",
 "algeria",


]

In [ ]:
# create pca df to visualize results in seaborn
geno_pca_df = pd.DataFrame({
    "PC1":PCs[:,0],
    "PC2":PCs[:,1],
    "label":geno_labels})

# create color palette for seaborn plot for labels
label_pal = {"vinifera":"#7570b3", 
             "wild":"#fb6a4a",
             "error":"#e6ab02",
             "MSU":"#66c2a5",
             "UC Davis":"#a6d854",
             "leaf1":"#67000d",
             "leaf2":"#a50f15",
             "leaf3":"#cb181d",
             "leaf4":"#ef3b2c",
             "rootstock":"orange",
             "dissected":"gold",
             "algeria":"#97D700"
            }

xlim1 = -0.35
xlim2 = 0.35
ylim1 = -0.15
ylim2 = 0.20


plt.figure(figsize=(10,10))
ax=sns.scatterplot(data=geno_pca_df, x="PC1", y="PC2", hue="label", palette=label_pal, zorder=2, s=50)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.xlim(xlim1,xlim2)
plt.ylim(ylim1,ylim2)
plt.grid()
plt.gca().set_aspect("equal")


In [ ]:
# recode labels as mature, leaf1, leaf2, leaf3, leaf4

devo_labels = []

for i in range(len(new_node_list)):
    
    if new_node_list[i][6:] == "mature":
        devo_labels.append("mature")
    elif int(new_node_list[i][6:]) <= 4:
        devo_labels.append("leaf" + str(int(new_node_list[i][-1])))
    elif int(new_node_list[i][6:]) > 4:
        devo_labels.append("mature")
        
devo_pal = { 
             "leaf1":"#F8485E",
             "leaf2":"#FF8F1C",
             "leaf3":"#FEDB00",
             "leaf4":"#97D700",
             "mature":"#1E22AA",
}


In [ ]:
proj_PCs_arr = np.zeros((10,100,2))

for i in range(np.shape(leaf_model_arr)[0]): # for each species
    
    for j in range(np.shape(leaf_model_arr)[1]): # for each modeled leaf
        
        curr_leaf = leaf_model_arr[i,j,:,:] # get current leaf
        reshape_lf = np.reshape(curr_leaf, (1,-1)) # reshape
        proj_PCs = pca.transform(reshape_lf) # get PCs
        proj_PCs_arr[i,j,:] = proj_PCs

In [ ]:
# create pca df to visualize results in seaborn
devo_pca_df = pd.DataFrame({
    "PC1":PCs[:,0],
    "PC2":PCs[:,1],
    "label":devo_labels})

xlim1 = -0.35
xlim2 = 0.35
ylim1 = -0.15
ylim2 = 0.20

plt.figure(figsize=(10,10))
ax=sns.scatterplot(data=devo_pca_df, x="PC1", y="PC2", hue="label", palette=devo_pal, zorder=2, s=70, alpha=0.7)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.xlim(xlim1,xlim2)
plt.ylim(ylim1,ylim2)
plt.grid()
plt.gca().set_aspect("equal")

for i in range(len(proj_PCs_arr)):
    
    plt.plot(proj_PCs_arr[i,:,0], proj_PCs_arr[i,:,1], lw=2)

In [ ]:
# Create PC values to reconstruct

numPC1 = 10 # set PC1 intervals
numPC2 = 5 # set PC2 intervals

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

In [ ]:
# Visualize the morphospace

cmap = cm.get_cmap('plasma', 100) # set the color map
s = 0.8 # set the scale

plt.figure(figsize=(20,20))

for i in PC1_vals:
    
    print(i)
    
    for j in PC2_vals:
        
        pc1_val = i
        pc2_val = j

        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_leaf = np.reshape(inv_leaf, (1672,2))

        inv_leaf_veinX = inv_leaf[0:num_vein_coords,0]
        inv_leaf_veinY = inv_leaf[0:num_vein_coords,1]
        inv_leaf_bladeX = inv_leaf[num_vein_coords:,0]
        inv_leaf_bladeY = inv_leaf[num_vein_coords:,1]

        vein_area = PolyArea(inv_leaf_veinX,inv_leaf_veinY)
        leaf_area = PolyArea(inv_leaf_bladeX,inv_leaf_bladeY)
        blade_area = leaf_area-vein_area
        vtb_ratio = np.log(vein_area/blade_area)
        vtb_min = -4
        norm_vtb = vtb_ratio/vtb_min
        hexcol = matplotlib.colors.rgb2hex(cmap(norm_vtb))

        plt.fill(inv_leaf_bladeX*s+pc1_val, inv_leaf_bladeY*s+pc2_val, c="lightgray")
        plt.plot(inv_leaf_bladeX*s+pc1_val, inv_leaf_bladeY*s+pc2_val, c="k", lw=1)
        plt.fill(inv_leaf_veinX*s+pc1_val, inv_leaf_veinY*s+pc2_val, c=hexcol)


ax=sns.scatterplot(data=geno_pca_df, x="PC1", y="PC2", hue="label", palette=label_pal, zorder=2, s=300, alpha=1)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

#plt.xlim(xlim1,xlim2)
#plt.ylim(ylim1,ylim2)
         
xlab = "PC1, " + str(np.round(pca.explained_variance_ratio_[0],3)*100) + "%"
ylab = "PC2, " + str(np.round(pca.explained_variance_ratio_[1],3)*100) + "%"
plt.xlabel(xlab, fontsize=24)
plt.ylabel(ylab, fontsize=24)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor("white")
plt.grid()
plt.gca().set_axisbelow(True)

plt.savefig("./images/Fig04C_geno_morphospace.png", bbox_inches='tight')


In [ ]:
# Visualize the morphospace

cmap = cm.get_cmap('plasma', 100) # set the color map
s = 0.8 # set the scale

plt.figure(figsize=(20,20))

for i in PC1_vals:
    
    print(i)
    
    for j in PC2_vals:
        
        pc1_val = i
        pc2_val = j

        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_leaf = np.reshape(inv_leaf, (1672,2))

        inv_leaf_veinX = inv_leaf[0:num_vein_coords,0]
        inv_leaf_veinY = inv_leaf[0:num_vein_coords,1]
        inv_leaf_bladeX = inv_leaf[num_vein_coords:,0]
        inv_leaf_bladeY = inv_leaf[num_vein_coords:,1]

        vein_area = PolyArea(inv_leaf_veinX,inv_leaf_veinY)
        leaf_area = PolyArea(inv_leaf_bladeX,inv_leaf_bladeY)
        blade_area = leaf_area-vein_area
        vtb_ratio = np.log(vein_area/blade_area)
        vtb_min = -4
        norm_vtb = vtb_ratio/vtb_min
        hexcol = matplotlib.colors.rgb2hex(cmap(norm_vtb))

        plt.fill(inv_leaf_bladeX*s+pc1_val, inv_leaf_bladeY*s+pc2_val, c="lightgray")
        plt.plot(inv_leaf_bladeX*s+pc1_val, inv_leaf_bladeY*s+pc2_val, c="k", lw=1)
        plt.fill(inv_leaf_veinX*s+pc1_val, inv_leaf_veinY*s+pc2_val, c=hexcol)


ax=sns.scatterplot(data=devo_pca_df, x="PC1", y="PC2", hue="label", palette=devo_pal, zorder=2, s=300, alpha=1)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

#plt.xlim(xlim1,xlim2)
#plt.ylim(ylim1,ylim2)
         
xlab = "PC1, " + str(np.round(pca.explained_variance_ratio_[0],3)*100) + "%"
ylab = "PC2, " + str(np.round(pca.explained_variance_ratio_[1],3)*100) + "%"
plt.xlabel(xlab, fontsize=24)
plt.ylabel(ylab, fontsize=24)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor("white")
plt.grid()
plt.gca().set_axisbelow(True)

for i in range(len(proj_PCs_arr)):
    
    plt.plot(proj_PCs_arr[i,:,0], proj_PCs_arr[i,:,1], lw=5)

plt.savefig("./images/Fig04D_devo_morphospace.png", bbox_inches='tight')

In [ ]:
# make plots for legends

cmap = cm.get_cmap('plasma', 100) # set the color map

min_vtb = 0.15785176146198115
max_vtb = 1.9468537604876768

delta = 0.4
steps = np.linspace(0, 1, 10)

for i in steps:
    
    print(i*-4)
    
    hexcol = matplotlib.colors.rgb2hex(cmap(i))
    plt.scatter(i/80,0, c=hexcol, s=200)


plt.savefig("./images/Fig04_vtb_legend.png", bbox_inches='tight')

In [ ]:
for i in range(10):
    plt.scatter(i/80,0,s=200,label=leaf_devo_list[i][0])
    
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.savefig("./images/Fig04_species_legend.png", bbox_inches='tight')

# LINEAR DISCRIMINANT ANALYSIS

The LinearDiscriminantAnalysis function from Scikit-learn was used for Linear Discriminant Analysis (LDA). The predict function was used on resulting models to classify leaf identities. The confusion_matrix function was used to calculate and visualize confusion matrices. 

#### By genotype

In [ ]:
# use the reshape function to flatten proc_arr to 2D
reshaped_arr = proc_arr.reshape(np.shape(proc_arr)[0], 
                                      np.shape(proc_arr)[1]*2) 

# create a df for LDA by genotype
geno_df = pd.DataFrame(data=reshaped_arr[:,:])

# add the genotype labels
geno_df["geno"] = geno_labels

# create input and output variables
X = geno_df.iloc[:,0:3344]
y = geno_df["geno"]

# fit the LDA model
geno_model = LinearDiscriminantAnalysis()
geno_model.fit(X,y)

# retrieve LDA scalings and coefficients
geno_scalings = geno_model.scalings_
geno_coefs = geno_model.coef_

# perform prediction
geno_prediction= geno_model.predict(X)
comparison_result = [X == y for X, y in zip(y, geno_prediction)]

count_true_pl = comparison_result.count(True)
count_false_pl = comparison_result.count(False)
print("Count of True values for genotype:", count_true_pl)
print("Count of False values for genotype:", count_false_pl)

# Create confusion matrix
true_geno_values = geno_labels
predicted_geno_values = geno_prediction

cm_geno = confusion_matrix(true_geno_values, predicted_geno_values)

classes = ["algeria","dissected","rootstock","vinifera","wild"]

plt.figure(figsize=(10,10))
sns.heatmap(cm_geno, annot=True, annot_kws={"fontsize":60}, fmt="d", cmap="viridis", square=True, cbar=False,xticklabels=classes,yticklabels=classes)
plt.savefig("./images/Fig05A_geno_cm.png", bbox_inches='tight')



In [ ]:
# find incorrectly predicted leaves

for i in range(len(species_list)):
    print(comparison_result[i], species_list[i])

In [ ]:
# create df to plot LDA results
data_plot = geno_model.fit(X, y).transform(X)
geno_plot_df = pd.DataFrame(data=data_plot[:,:])
geno_plot_df["geno"] = geno_labels
geno_plot_df.columns = ["LD1","LD2","LD3","LD4","geno"]

In [ ]:
# plot LDA
plt.figure(figsize=(10,10))
ax=sns.scatterplot(data=geno_plot_df, x="LD1", y="LD2", hue="geno", palette=label_pal, zorder=2, s=220, alpha=0.7)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.grid()
plt.savefig("./images/Fig05B_geno_LDA_panel1.png", bbox_inches='tight')


In [ ]:
# plot LDA
plt.figure(figsize=(10,10))
ax=sns.scatterplot(data=geno_plot_df, x="LD2", y="LD3", hue="geno", palette=label_pal, zorder=2, s=220, alpha=0.7)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.grid()
plt.savefig("./images/Fig05C_geno_LDA_panel2.png", bbox_inches='tight')


In [ ]:
# visualize average leaf shapes for each class

vinifera_list = []
dissected_list = []
wild_list = []
rootstock_list = []
algeria_list = []

for i in range(len(geno_labels)):
    
    if geno_labels[i]=="vinifera":
        vinifera_list.append(i)
    elif geno_labels[i]=="dissected":
        dissected_list.append(i)
    elif geno_labels[i]=="wild":
        wild_list.append(i)
    elif geno_labels[i]=="rootstock":
        rootstock_list.append(i)
    elif geno_labels[i]=="algeria":
        algeria_list.append(i)
        
mean_vinifera = np.mean(proc_arr[vinifera_list,:,:],0)
mean_dissected = np.mean(proc_arr[dissected_list,:,:],0)
mean_wild = np.mean(proc_arr[wild_list,:,:],0)
mean_rootstock = np.mean(proc_arr[rootstock_list,:,:],0)
mean_algeria = np.mean(proc_arr[algeria_list,:,:],0)

plt.figure(figsize=(20,20))

ax1 = plt.subplot(1,5,1)
ax1.plot(mean_vinifera[num_vein_coords:,0], mean_vinifera[num_vein_coords:,1], c="k") # blade
ax1.fill(mean_vinifera[num_vein_coords:,0], mean_vinifera[num_vein_coords:,1], c="#7570b3", alpha=0.2) # blade
ax1.fill(mean_vinifera[0:num_vein_coords,0], mean_vinifera[0:num_vein_coords,1], c="#7570b3") # veins
ax1.set_aspect("equal")
ax1.axis("off")

ax2 = plt.subplot(1,5,2)
ax2.plot(mean_rootstock[num_vein_coords:,0], mean_rootstock[num_vein_coords:,1], c="k") # blade
ax2.fill(mean_rootstock[num_vein_coords:,0], mean_rootstock[num_vein_coords:,1], c="orange", alpha=0.2) # blade
ax2.fill(mean_rootstock[0:num_vein_coords,0], mean_rootstock[0:num_vein_coords,1], c="orange") # veins
ax2.set_aspect("equal")
ax2.axis("off")

ax3 = plt.subplot(1,5,3)
ax3.plot(mean_wild[num_vein_coords:,0], mean_wild[num_vein_coords:,1], c="k") # blade
ax3.fill(mean_wild[num_vein_coords:,0], mean_wild[num_vein_coords:,1], c="#fb6a4a", alpha=0.2) # blade
ax3.fill(mean_wild[0:num_vein_coords,0], mean_wild[0:num_vein_coords,1], c="#fb6a4a") # veins
ax3.set_aspect("equal")
ax3.axis("off")

ax4 = plt.subplot(1,5,4)
ax4.plot(mean_dissected[num_vein_coords:,0], mean_dissected[num_vein_coords:,1], c="k") # blade
ax4.fill(mean_dissected[num_vein_coords:,0], mean_dissected[num_vein_coords:,1], c="gold", alpha=0.2) # blade
ax4.fill(mean_dissected[0:num_vein_coords,0], mean_dissected[0:num_vein_coords,1], c="gold") # veins
ax4.set_aspect("equal")
ax4.axis("off")

ax5 = plt.subplot(1,5,5)
ax5.plot(mean_algeria[num_vein_coords:,0], mean_algeria[num_vein_coords:,1], c="k") # blade
ax5.fill(mean_algeria[num_vein_coords:,0], mean_algeria[num_vein_coords:,1], c="#97D700", alpha=0.2) # blade
ax5.fill(mean_algeria[0:num_vein_coords,0], mean_algeria[0:num_vein_coords,1], c="#97D700") # veins
ax5.set_aspect("equal")
ax5.axis("off")

plt.tight_layout()
plt.savefig("./images/Fig05D_geno_average_leaves.png", bbox_inches='tight')

In [ ]:
# Create PC values to reconstruct

numPC1 = 20 # set PC1 intervals
numPC2 = 10 # set PC2 intervals

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

In [ ]:
# Visualize the morphospace

s = 0.4 # set the scale
prob_cutoff = 0.95

plt.figure(figsize=(20,20))

for i in PC1_vals:
    
    print(i)
    
    for j in PC2_vals:
        
        pc1_val = i
        pc2_val = j

        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_prob = np.reshape(inv_leaf, (1, -1))
        
        # for predicting identity of the leaf
        leaf_probs = geno_model.predict_proba(inv_prob)
        
        if np.max(leaf_probs) < prob_cutoff:
            leaf_pred = "none"
            leaf_col = "lightgray"
            
        elif np.max(leaf_probs) >= prob_cutoff:
            prob_ind = np.argmax(leaf_probs)
            
        if prob_ind==0:
            leaf_pred = "algeria"
            leaf_col = "#97D700"
        elif prob_ind==1:
            leaf_pred = "dissected"
            leaf_col = "gold"
        elif prob_ind==2:
            leaf_pred = "rootstock"
            leaf_col = "orange"
        elif prob_ind==3:
            leaf_pred = "vinifera"
            leaf_col = "#7570b3"
        elif prob_ind==4:
            leaf_pred = "wild"
            leaf_col = "#fb6a4a"
        
        # for plotting the leaf
        inv_leaf = np.reshape(inv_leaf, (1672,2))

        inv_leaf_veinX = inv_leaf[0:num_vein_coords,0]
        inv_leaf_veinY = inv_leaf[0:num_vein_coords,1]
        inv_leaf_bladeX = inv_leaf[num_vein_coords:,0]
        inv_leaf_bladeY = inv_leaf[num_vein_coords:,1]

        vein_area = PolyArea(inv_leaf_veinX,inv_leaf_veinY)
        leaf_area = PolyArea(inv_leaf_bladeX,inv_leaf_bladeY)
        blade_area = leaf_area-vein_area
        vtb_ratio = np.log(vein_area/blade_area)
        vtb_min = -4
        norm_vtb = vtb_ratio/vtb_min
        hexcol = matplotlib.colors.rgb2hex(cmap(norm_vtb))

        plt.fill(inv_leaf_bladeX*s+pc1_val, inv_leaf_bladeY*s+pc2_val, c=leaf_col, alpha=0.2)
        plt.plot(inv_leaf_bladeX*s+pc1_val, inv_leaf_bladeY*s+pc2_val, c="k", lw=1)
        plt.fill(inv_leaf_veinX*s+pc1_val, inv_leaf_veinY*s+pc2_val, c=leaf_col, zorder=2)


sns.scatterplot(data=geno_pca_df, x="PC1", y="PC2", color="black", s=100, zorder=3)

#plt.xlim(xlim1,xlim2)
#plt.ylim(ylim1,ylim2)
         
xlab = "PC1, " + str(np.round(pca.explained_variance_ratio_[0],3)*100) + "%"
ylab = "PC2, " + str(np.round(pca.explained_variance_ratio_[1],3)*100) + "%"
plt.xlabel(xlab, fontsize=24)
plt.ylabel(ylab, fontsize=24)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor("white")
plt.grid()
plt.gca().set_axisbelow(True)

hull = ConvexHull(PCs)

for simplex in hull.simplices:
        plt.plot(hull.points[simplex, 0], hull.points[simplex, 1], '-k')
        
plt.savefig("./images/Fig05E_geno_prediction_PCA.png", bbox_inches='tight')

#### By development

In [ ]:
# recode labels as mature, leaf1, leaf2, leaf3, leaf4

devo_labels = []

for i in range(len(new_node_list)):
    
    if new_node_list[i][6:] == "mature":
        devo_labels.append("mature")
    elif int(new_node_list[i][6:]) <= 4:
        devo_labels.append("leaf" + str(int(new_node_list[i][-1])))
    elif int(new_node_list[i][6:]) > 4:
        devo_labels.append("mature")


In [ ]:
# use the reshape function to flatten proc_arr to 2D
reshaped_arr = proc_arr.reshape(np.shape(proc_arr)[0], 
                                      np.shape(proc_arr)[1]*2) 

# create a df for LDA by genotype
devo_df = pd.DataFrame(data=reshaped_arr[:,:])

# add the development labels
devo_df["devo"] = devo_labels

# create input and output variables
X = devo_df.iloc[:,0:3344]
y = devo_df["devo"]

# fit the LDA model
devo_model = LinearDiscriminantAnalysis()
devo_model.fit(X,y)

# retrieve LDA scalings and coefficients
devo_scalings = devo_model.scalings_
devo_coefs = devo_model.coef_

# perform prediction
devo_prediction= devo_model.predict(X)
comparison_result = [X == y for X, y in zip(y, devo_prediction)]

count_true_pl = comparison_result.count(True)
count_false_pl = comparison_result.count(False)
print("Count of True values by node:", count_true_pl)
print("Count of False values by node:", count_false_pl)

# Create confusion matrix
true_devo_values = devo_labels
predicted_devo_values = devo_prediction

cm_devo = confusion_matrix(true_devo_values, predicted_devo_values)

classes = ["leaf1","leaf2","leaf3","leaf4","mature"]

plt.figure(figsize=(10,10))
sns.heatmap(cm_devo, annot=True,annot_kws={"fontsize":60}, fmt="d", cmap="viridis", square=True, cbar=False, xticklabels=classes,yticklabels=classes)
plt.savefig("./images/Fig06A_devo_cm.png", bbox_inches='tight')


In [ ]:
# create df to plot LDA results
data_plot = devo_model.fit(X, y).transform(X)
devo_plot_df = pd.DataFrame(data=data_plot[:,:])
devo_plot_df["devo"] = devo_labels
devo_plot_df.columns = ["LD1","LD2","LD3","LD4","devo"]

In [ ]:
np.shape(leaf_model_arr)

In [ ]:
i = 2
j = 50

lf_LD_vals = np.zeros((1000,4)) # for 1000 modeled leaves store 4 LD vals
lf_pred_vals = []

counter = 0

for i in range(10): # for 10 species
    for j in range(100): # for 100 modeled leaves

        lf = leaf_model_arr[i,j,:,:] # get the current leaf
        lf = np.reshape(lf,(1,-1)) # reshape

        lf_LDs = devo_model.transform(lf) # get leaf LDs
        lf_pred = devo_model.predict(lf)[0] # predict leaf node
        
        lf_LD_vals[counter,:] = lf_LDs
        lf_pred_vals.append(lf_pred)
                   
        counter+=1
        

In [ ]:
# plot LDA

plt.figure(figsize=(10,10))

ax=sns.scatterplot(data=devo_plot_df, x="LD1", y="LD2", hue="devo", palette=devo_pal, zorder=2, s=220, alpha=0.7)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.grid()

for i in range(len(lf_LD_vals)):
    
    nd = lf_pred_vals[i] # get node
    
    if nd == "leaf1":
        lf_col = "#F8485E"
    elif nd == "leaf2":
        lf_col = "#FF8F1C"
    elif nd == "leaf3":
        lf_col = "#FEDB00"
    elif nd == "leaf4":
        lf_col = "#97D700"
    elif nd == "mature":
        lf_col = "#1E22AA"
    
    plt.scatter(lf_LD_vals[i,0],lf_LD_vals[i,1], s=3, c=lf_col)
    
plt.savefig("./images/Fig06B_devo_LDA_panel1.png", bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,10))

ax=sns.scatterplot(data=devo_plot_df, x="LD3", y="LD4", hue="devo", palette=devo_pal, zorder=2, s=220, alpha=0.7)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.grid()

for i in range(len(lf_LD_vals)):
    
    nd = lf_pred_vals[i] # get node
    
    if nd == "leaf1":
        lf_col = "#F8485E"
    elif nd == "leaf2":
        lf_col = "#FF8F1C"
    elif nd == "leaf3":
        lf_col = "#FEDB00"
    elif nd == "leaf4":
        lf_col = "#97D700"
    elif nd == "mature":
        lf_col = "#1E22AA"
    
    plt.scatter(lf_LD_vals[i,2],lf_LD_vals[i,3], s=3, c=lf_col)
    
plt.savefig("./images/Fig06C_devo_LDA_panel2.png", bbox_inches='tight')

In [ ]:
# visualize average leaf shapes for each class

leaf1_list = []
leaf2_list = []
leaf3_list = []
leaf4_list = []
mature_list = []

for i in range(len(devo_labels)):
    
    if devo_labels[i]=="leaf1":
        leaf1_list.append(i)
    elif devo_labels[i]=="leaf2":
        leaf2_list.append(i)
    elif devo_labels[i]=="leaf3":
        leaf3_list.append(i)
    elif devo_labels[i]=="leaf4":
        leaf4_list.append(i)
    elif devo_labels[i]=="mature":
        mature_list.append(i)
        
mean_leaf1 = np.mean(proc_arr[leaf1_list,:,:],0)
mean_leaf2 = np.mean(proc_arr[leaf2_list,:,:],0)
mean_leaf3 = np.mean(proc_arr[leaf3_list,:,:],0)
mean_leaf4 = np.mean(proc_arr[leaf4_list,:,:],0)
mean_mature = np.mean(proc_arr[mature_list,:,:],0)

plt.figure(figsize=(20,20))


ax1 = plt.subplot(1,5,1)
ax1.plot(mean_leaf1[num_vein_coords:,0], mean_leaf1[num_vein_coords:,1], c="k") # blade
ax1.fill(mean_leaf1[num_vein_coords:,0], mean_leaf1[num_vein_coords:,1], c="#F8485E", alpha=0.2) # blade
ax1.fill(mean_leaf1[0:num_vein_coords,0], mean_leaf1[0:num_vein_coords,1], c="#F8485E") # veins
ax1.set_aspect("equal")
ax1.axis("off")

ax2 = plt.subplot(1,5,2)
ax2.plot(mean_leaf2[num_vein_coords:,0], mean_leaf2[num_vein_coords:,1], c="k") # blade
ax2.fill(mean_leaf2[num_vein_coords:,0], mean_leaf2[num_vein_coords:,1], c="#FF8F1C", alpha=0.2) # blade
ax2.fill(mean_leaf2[0:num_vein_coords,0], mean_leaf2[0:num_vein_coords,1], c="#FF8F1C") # veins
ax2.set_aspect("equal")
ax2.axis("off")

ax3 = plt.subplot(1,5,3)
ax3.plot(mean_leaf3[num_vein_coords:,0], mean_leaf3[num_vein_coords:,1], c="k") # blade
ax3.fill(mean_leaf3[num_vein_coords:,0], mean_leaf3[num_vein_coords:,1], c="#FEDB00", alpha=0.2) # blade
ax3.fill(mean_leaf3[0:num_vein_coords,0], mean_leaf3[0:num_vein_coords,1], c="#FEDB00") # veins
ax3.set_aspect("equal")
ax3.axis("off")

ax4 = plt.subplot(1,5,4)
ax4.plot(mean_leaf4[num_vein_coords:,0], mean_leaf4[num_vein_coords:,1], c="k") # blade
ax4.fill(mean_leaf4[num_vein_coords:,0], mean_leaf4[num_vein_coords:,1], c="#97D700", alpha=0.2) # blade
ax4.fill(mean_leaf4[0:num_vein_coords,0], mean_leaf4[0:num_vein_coords,1], c="#97D700") # veins
ax4.set_aspect("equal")
ax4.axis("off")

ax5 = plt.subplot(1,5,5)
ax5.plot(mean_mature[num_vein_coords:,0], mean_mature[num_vein_coords:,1], c="k") # blade
ax5.fill(mean_mature[num_vein_coords:,0], mean_mature[num_vein_coords:,1], c="#1E22AA", alpha=0.2) # blade
ax5.fill(mean_mature[0:num_vein_coords,0], mean_mature[0:num_vein_coords,1], c="#1E22AA") # veins
ax5.set_aspect("equal")
ax5.axis("off")

plt.tight_layout()

plt.savefig("./images/Fig06D_devo_average_leaves.png", bbox_inches='tight')

In [ ]:
# Create PC values to reconstruct

numPC1 = 20 # set PC1 intervals
numPC2 = 10 # set PC2 intervals

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

In [ ]:
# Visualize the morphospace

s = 0.4 # set the scale
prob_cutoff = 0.95

plt.figure(figsize=(20,20))

for i in PC1_vals:
    
    print(i)
    
    for j in PC2_vals:
        
        pc1_val = i
        pc2_val = j

        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_prob = np.reshape(inv_leaf, (1, -1))
        
        # for predicting identity of the leaf
        leaf_probs = devo_model.predict_proba(inv_prob)
        
        if np.max(leaf_probs) < prob_cutoff:
            leaf_pred = "none"
            leaf_col = "lightgray"
            
        elif np.max(leaf_probs) >= prob_cutoff:
            prob_ind = np.argmax(leaf_probs)
  

            
        if prob_ind==0:
            leaf_pred = "leaf1"
            leaf_col = "#F8485E"
        elif prob_ind==1:
            leaf_pred = "leaf2"
            leaf_col = "#FF8F1C"
        elif prob_ind==2:
            leaf_pred = "leaf3"
            leaf_col = "#FEDB00"
        elif prob_ind==3:
            leaf_pred = "leaf4"
            leaf_col = "#97D700"
        elif prob_ind==4:
            leaf_pred = "mature"
            leaf_col = "#1E22AA"
        
        
        # for plotting the leaf
        inv_leaf = np.reshape(inv_leaf, (1672,2))

        inv_leaf_veinX = inv_leaf[0:num_vein_coords,0]
        inv_leaf_veinY = inv_leaf[0:num_vein_coords,1]
        inv_leaf_bladeX = inv_leaf[num_vein_coords:,0]
        inv_leaf_bladeY = inv_leaf[num_vein_coords:,1]

        vein_area = PolyArea(inv_leaf_veinX,inv_leaf_veinY)
        leaf_area = PolyArea(inv_leaf_bladeX,inv_leaf_bladeY)
        blade_area = leaf_area-vein_area
        vtb_ratio = np.log(vein_area/blade_area)
        vtb_min = -4
        norm_vtb = vtb_ratio/vtb_min
        hexcol = matplotlib.colors.rgb2hex(cmap(norm_vtb))

        plt.fill(inv_leaf_bladeX*s+pc1_val, inv_leaf_bladeY*s+pc2_val, c=leaf_col, alpha=0.2)
        plt.plot(inv_leaf_bladeX*s+pc1_val, inv_leaf_bladeY*s+pc2_val, c="k", lw=1)
        plt.fill(inv_leaf_veinX*s+pc1_val, inv_leaf_veinY*s+pc2_val, c=leaf_col, zorder=2)


sns.scatterplot(data=devo_pca_df, x="PC1", y="PC2", color="black", s=100, zorder=3)

#plt.xlim(xlim1,xlim2)
#plt.ylim(ylim1,ylim2)
         
xlab = "PC1, " + str(np.round(pca.explained_variance_ratio_[0],3)*100) + "%"
ylab = "PC2, " + str(np.round(pca.explained_variance_ratio_[1],3)*100) + "%"
plt.xlabel(xlab, fontsize=24)
plt.ylabel(ylab, fontsize=24)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor("white")
plt.grid()
plt.gca().set_axisbelow(True)

for simplex in hull.simplices:
        plt.plot(hull.points[simplex, 0], hull.points[simplex, 1], '-k')
        
plt.savefig("./images/Fig06E_devo_prediction_PCA.png", bbox_inches='tight')

# Convex hull

To calculate a convex hull in a 9 dimensional principal component space, the ConvexHull function from SciPy was used followed by the Delaunay function to find simplices. A uniform distribution of points was sampled using the Dirichlet distribution using the SciPy dirichlet function.

In [ ]:
# From: https://stackoverflow.com/questions/59073952/how-to-get-uniformly-distributed-points-in-convex-hull
# Accessed 11 February 2024

def dist_in_hull(points, n):
    dims = points.shape[-1]
    hull = points[ConvexHull(points).vertices]
    deln = hull[Delaunay(hull).simplices]

    vols = np.abs(det(deln[:, :dims, :] - deln[:, dims:, :])) / np.math.factorial(dims)    
    sample = np.random.choice(len(vols), size = n, p = vols / vols.sum())

    return np.einsum('ijk, ij -> ik', deln[sample], dirichlet.rvs([1]*(dims + 1), size = n))

######
PC_NUMBER = 9
#######

# use the reshape function to flatten proc_arr to 2D
reshaped_arr = proc_arr.reshape(np.shape(proc_arr)[0], 
                                      np.shape(proc_arr)[1]*2) 

pca = PCA(n_components=PC_NUMBER) 
PCs = pca.fit_transform(reshaped_arr) # fit a PCA


In [ ]:
random_leaves = dist_in_hull(PCs,500)

In [ ]:
np.shape(random_leaves)

In [ ]:
# Visualize random genotype leaves


plt.figure(figsize=(20,25))

cmap = cm.get_cmap('plasma', 100) # set the color map
counter = 1
prob_cutoff = 0.95

for i in range(500):
    
    if i%50==0:
        print(i)

    inv_leaf = pca.inverse_transform(random_leaves[i,:])
    inv_leaf = np.reshape(inv_leaf, (1672,2))

    inv_leaf_veinX = inv_leaf[0:num_vein_coords,0]
    inv_leaf_veinY = inv_leaf[0:num_vein_coords,1]
    inv_leaf_bladeX = inv_leaf[num_vein_coords:,0]
    inv_leaf_bladeY = inv_leaf[num_vein_coords:,1]
    
    vein_area = PolyArea(inv_leaf_veinX,inv_leaf_veinY)
    leaf_area = PolyArea(inv_leaf_bladeX,inv_leaf_bladeY)
    blade_area = leaf_area-vein_area
    vtb_ratio = np.log(vein_area/blade_area)
    vtb_min = -4
    norm_vtb = vtb_ratio/vtb_min
    hexcol = matplotlib.colors.rgb2hex(cmap(norm_vtb))

    plt.subplot(25,20,counter)
    plt.plot(inv_leaf_bladeX, inv_leaf_bladeY, c="k", lw=1)
    plt.fill(inv_leaf_bladeX, inv_leaf_bladeY, c="lightgray")
    plt.fill(inv_leaf_veinX, inv_leaf_veinY, c=hexcol, zorder=2)
    plt.gca().set_aspect("equal")
    plt.gca().set_facecolor("white")
    plt.axis("off")

    counter += 1
        
plt.tight_layout()
plt.savefig("./images/Fig07_rand_leaves.png", bbox_inches='tight')

______

# Algeria analysis

____________

Get the names of just the varieties that were used

In [ ]:
# get names of all varieties, removing indices in the name
name_list = [] # a list to store file names
for i in range(len(index_list)): # for each index
    name_list.append(leaf_list[index_list[i]][4:-5]) # store the variety name

# get indices of Algerian leaves
algeria_indices = geno_pca_df[geno_pca_df["label"]=="algeria"].index

# create an array of Procrustes leaves just for Algeria
algeria_arr = proc_arr[algeria_indices]

# create a list of just the Algerian names
algeria_names = [] # to store algerian variety names
for i in algeria_indices:
    algeria_names.append(name_list[i])

## Average Algerian leaves

In [ ]:
# set Algerian leaf color palatte
a_cols = [
    "#66c2a5",#'AHMEUR BOU AHMEUR'
    "#fc8d62",#'BABARI'
    "#8da0cb",#'GENOTYPE 1'
    "#e78ac3",#'GENOTYPE 2'
    "#a6d854",#'GENOTYPE 3'
    "#ffd92f",#'GENOTYPE 4'
    "#e5c494",#'LOUALI'
    "#b3b3b3",#'TIZI OUININE'
]

In [ ]:
# visualize average leaf shapes for each genotype

ahmeur_list = []
babari_list = []
geno1_list = []
geno2_list = []
geno3_list = []
geno4_list = []
louali_list = []
tizi_list = []

for i in range(len(algeria_names)):
    
    if algeria_names[i]=='AHMEUR BOU AHMEUR':
        ahmeur_list.append(i)
    elif algeria_names[i]=='BABARI':
        babari_list.append(i)
    elif algeria_names[i]=='GENOTYPE 1':
        geno1_list.append(i)
    elif algeria_names[i]=='GENOTYPE 2':
        geno2_list.append(i)
    elif algeria_names[i]=='GENOTYPE 3':
        geno3_list.append(i)
    elif algeria_names[i]=='GENOTYPE 4':
        geno4_list.append(i)
    elif algeria_names[i]=='LOUALI':
        louali_list.append(i)
    elif algeria_names[i]=='TIZI OUININE':
        tizi_list.append(i)

mean_ahmeur = np.mean(algeria_arr[ahmeur_list,:,:],0)
mean_babari = np.mean(algeria_arr[babari_list,:,:],0)
mean_geno1 = np.mean(algeria_arr[geno1_list,:,:],0)
mean_geno2 = np.mean(algeria_arr[geno2_list,:,:],0)
mean_geno3 = np.mean(algeria_arr[geno3_list,:,:],0)
mean_geno4 = np.mean(algeria_arr[geno4_list,:,:],0)
mean_louali = np.mean(algeria_arr[louali_list,:,:],0)
mean_tizi = np.mean(algeria_arr[tizi_list,:,:],0)

plt.figure(figsize=(20,10))

ax1 = plt.subplot(2,4,1)
ax1.plot(mean_ahmeur[num_vein_coords:,0], mean_ahmeur[num_vein_coords:,1], c="k") # blade
ax1.fill(mean_ahmeur[num_vein_coords:,0], mean_ahmeur[num_vein_coords:,1], c=a_cols[0], alpha=0.2) # blade
ax1.fill(mean_ahmeur[0:num_vein_coords,0], mean_ahmeur[0:num_vein_coords,1], c=a_cols[0]) # veins
ax1.title.set_text('AHMEUR BOU AHMEUR')
ax1.set_aspect("equal")
ax1.axis("off")

ax2 = plt.subplot(2,4,2)
ax2.plot(mean_babari[num_vein_coords:,0], mean_babari[num_vein_coords:,1], c="k") # blade
ax2.fill(mean_babari[num_vein_coords:,0], mean_babari[num_vein_coords:,1], c=a_cols[1], alpha=0.2) # blade
ax2.fill(mean_babari[0:num_vein_coords,0], mean_babari[0:num_vein_coords,1], c=a_cols[1]) # veins
ax2.title.set_text('BABARI')
ax2.set_aspect("equal")
ax2.axis("off")

ax3 = plt.subplot(2,4,3)
ax3.plot(mean_geno1[num_vein_coords:,0], mean_geno1[num_vein_coords:,1], c="k") # blade
ax3.fill(mean_geno1[num_vein_coords:,0], mean_geno1[num_vein_coords:,1], c=a_cols[2], alpha=0.2) # blade
ax3.fill(mean_geno1[0:num_vein_coords,0], mean_geno1[0:num_vein_coords,1], c=a_cols[2]) # veins
ax3.title.set_text('GENOTYPE 1')
ax3.set_aspect("equal")
ax3.axis("off")

ax4 = plt.subplot(2,4,4)
ax4.plot(mean_geno2[num_vein_coords:,0], mean_geno2[num_vein_coords:,1], c="k") # blade
ax4.fill(mean_geno2[num_vein_coords:,0], mean_geno2[num_vein_coords:,1], c=a_cols[3], alpha=0.2) # blade
ax4.fill(mean_geno2[0:num_vein_coords,0], mean_geno2[0:num_vein_coords,1], c=a_cols[3]) # veins
ax4.title.set_text('GENOTYPE 2')
ax4.set_aspect("equal")
ax4.axis("off")

ax5 = plt.subplot(2,4,5)
ax5.plot(mean_geno3[num_vein_coords:,0], mean_geno3[num_vein_coords:,1], c="k") # blade
ax5.fill(mean_geno3[num_vein_coords:,0], mean_geno3[num_vein_coords:,1], c=a_cols[4], alpha=0.2) # blade
ax5.fill(mean_geno3[0:num_vein_coords,0], mean_geno3[0:num_vein_coords,1], c=a_cols[4]) # veins
ax5.title.set_text('GENOTYPE 3')
ax5.set_aspect("equal")
ax5.axis("off")

ax6 = plt.subplot(2,4,6)
ax6.plot(mean_geno4[num_vein_coords:,0], mean_geno4[num_vein_coords:,1], c="k") # blade
ax6.fill(mean_geno4[num_vein_coords:,0], mean_geno4[num_vein_coords:,1], c=a_cols[5], alpha=0.2) # blade
ax6.fill(mean_geno4[0:num_vein_coords,0], mean_geno4[0:num_vein_coords,1], c=a_cols[5]) # veins
ax6.title.set_text('GENOTYPE 4')
ax6.set_aspect("equal")
ax6.axis("off")

ax7 = plt.subplot(2,4,7)
ax7.plot(mean_louali[num_vein_coords:,0], mean_louali[num_vein_coords:,1], c="k") # blade
ax7.fill(mean_louali[num_vein_coords:,0], mean_louali[num_vein_coords:,1], c=a_cols[6], alpha=0.2) # blade
ax7.fill(mean_louali[0:num_vein_coords,0], mean_louali[0:num_vein_coords,1], c=a_cols[6]) # veins
ax7.title.set_text('LOUALI')
ax7.set_aspect("equal")
ax7.axis("off")

ax8 = plt.subplot(2,4,8)
ax8.plot(mean_tizi[num_vein_coords:,0], mean_tizi[num_vein_coords:,1], c="k") # blade
ax8.fill(mean_tizi[num_vein_coords:,0], mean_tizi[num_vein_coords:,1], c=a_cols[7], alpha=0.2) # blade
ax8.fill(mean_tizi[0:num_vein_coords,0], mean_tizi[0:num_vein_coords,1], c=a_cols[7]) # veins
ax8.title.set_text('TIZI OUININE')
ax8.set_aspect("equal")
ax8.axis("off")

plt.tight_layout()

## Principal component analysis Algerian leaves

In [ ]:
######
PC_NUMBER = 2
#######

# use the reshape function to flatten proc_arr to 2D
reshaped_arr = algeria_arr.reshape(np.shape(algeria_arr)[0], 
                                      np.shape(algeria_arr)[1]*2) 

pca = PCA(n_components=PC_NUMBER) 
PCs = pca.fit_transform(reshaped_arr) # fit a PCA

print(pca.explained_variance_ratio_) # print out explained variance for each PC
print(pca.explained_variance_ratio_.cumsum())

In [ ]:
# Create dataframe for Algeria PCA

algeria_pca_df = pd.DataFrame({
    "PC1":PCs[:,0],
    "PC2":PCs[:,1],
    "label":algeria_names
})

# Create color palette for seaborn

label_pal = {
    'AHMEUR BOU AHMEUR':"#66c2a5",
    'BABARI':"#fc8d62",
    'GENOTYPE 1':"#8da0cb",
    'GENOTYPE 2':"#e78ac3",
    'GENOTYPE 3':"#a6d854",
    'GENOTYPE 4':"#ffd92f",
    'LOUALI':"#e5c494",
    'TIZI OUININE':"#b3b3b3",
}

In [ ]:
# Create PC values to reconstruct

numPC1 = 10 # set PC1 intervals
numPC2 = 7 # set PC2 intervals

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

In [ ]:
# Visualize the morphospace

cmap = cm.get_cmap('plasma', 100) # set the color map
s = 0.45 # set the scale

plt.figure(figsize=(20,20))

for i in PC1_vals:
    
    print(i)
    
    for j in PC2_vals:
        
        pc1_val = i
        pc2_val = j

        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_leaf = np.reshape(inv_leaf, (1672,2))

        inv_leaf_veinX = inv_leaf[0:num_vein_coords,0]
        inv_leaf_veinY = inv_leaf[0:num_vein_coords,1]
        inv_leaf_bladeX = inv_leaf[num_vein_coords:,0]
        inv_leaf_bladeY = inv_leaf[num_vein_coords:,1]

        vein_area = PolyArea(inv_leaf_veinX,inv_leaf_veinY)
        leaf_area = PolyArea(inv_leaf_bladeX,inv_leaf_bladeY)
        blade_area = leaf_area-vein_area
        vtb_ratio = np.log(vein_area/blade_area)
        vtb_min = -4
        norm_vtb = vtb_ratio/vtb_min
        hexcol = matplotlib.colors.rgb2hex(cmap(norm_vtb))

        plt.fill(inv_leaf_bladeX*s+pc1_val, inv_leaf_bladeY*s+pc2_val, c="lightgray",alpha=0.3)
        plt.plot(inv_leaf_bladeX*s+pc1_val, inv_leaf_bladeY*s+pc2_val, c="darkgray", lw=1, alpha=0.6)
        plt.fill(inv_leaf_veinX*s+pc1_val, inv_leaf_veinY*s+pc2_val, c="darkgray",alpha=0.6)


ax=sns.scatterplot(data=algeria_pca_df, x="PC1", y="PC2", hue="label", palette=label_pal, zorder=2, s=500, alpha=1)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

#plt.xlim(xlim1,xlim2)
#plt.ylim(ylim1,ylim2)
         
xlab = "PC1, " + str(np.round(pca.explained_variance_ratio_[0],3)*100) + "%"
ylab = "PC2, " + str(np.round(pca.explained_variance_ratio_[1],3)*100) + "%"
plt.xlabel(xlab, fontsize=24)
plt.ylabel(ylab, fontsize=24)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor("white")
plt.grid()
plt.gca().set_axisbelow(True)

## Linear Discriminant Analysis Algerian leaves

In [ ]:
# use the reshape function to flatten algeria_arr to 2D
reshaped_arr = algeria_arr.reshape(np.shape(algeria_arr)[0], 
                                      np.shape(algeria_arr)[1]*2) 

# create a df for LDA by genotype
geno_df = pd.DataFrame(data=reshaped_arr[:,:])

# add the genotype labels
geno_df["geno"] = algeria_names

# create input and output variables
X = geno_df.iloc[:,0:3344]
y = geno_df["geno"]

# fit the LDA model
geno_model = LinearDiscriminantAnalysis()
geno_model.fit(X,y)

# retrieve LDA scalings and coefficients
geno_scalings = geno_model.scalings_
geno_coefs = geno_model.coef_

# perform prediction
geno_prediction= geno_model.predict(X)
comparison_result = [X == y for X, y in zip(y, geno_prediction)]

count_true_pl = comparison_result.count(True)
count_false_pl = comparison_result.count(False)
print("Count of True values for genotype:", count_true_pl)
print("Count of False values for genotype:", count_false_pl)

# Create confusion matrix
true_geno_values = algeria_names
predicted_geno_values = geno_prediction

cm_geno = confusion_matrix(true_geno_values, predicted_geno_values)

classes = ['AHMEUR BOU AHMEUR', 'BABARI', 'GENOTYPE 1', 'GENOTYPE 2',
       'GENOTYPE 3', 'GENOTYPE 4', 'LOUALI', 'TIZI OUININE']

plt.figure(figsize=(10,10))
sns.heatmap(cm_geno, annot=True, annot_kws={"fontsize":30}, fmt="d", cmap="viridis", square=True, cbar=False,xticklabels=classes,yticklabels=classes)


In [ ]:
# create df to plot LDA results
data_plot = geno_model.fit(X, y).transform(X)
geno_plot_df = pd.DataFrame(data=data_plot[:,:])
geno_plot_df["geno"] = algeria_names
geno_plot_df.columns = ["LD1","LD2","LD3","LD4","LD5","LD6","LD7","geno"]

In [ ]:
# plot LDA
plt.figure(figsize=(20,6))
pt_size=200

plt.subplot(1,3,1)
sns.scatterplot(data=geno_plot_df, x="LD1", y="LD2", hue="geno", palette=label_pal, zorder=2, s=pt_size, alpha=0.7, legend=False)
plt.grid()

plt.subplot(1,3,2)
sns.scatterplot(data=geno_plot_df, x="LD3", y="LD4", hue="geno", palette=label_pal, zorder=2, s=pt_size, alpha=0.7, legend=False)
plt.grid()

plt.subplot(1,3,3)
sns.scatterplot(data=geno_plot_df, x="LD5", y="LD6", hue="geno", palette=label_pal, zorder=2, s=pt_size, alpha=0.7, legend=False)
plt.grid()

plt.tight_layout()